<a href="https://colab.research.google.com/github/23013093uhi/23013093_DataAnalytics/blob/main/23013093_assignment1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Corinn Sinnott
###23013093
###Assignment1-2

In [1]:
#load packages
# pandas is used to create the data frame (McKinney et al., 2010)
import pandas as pd

# numpy is used for calculations (Harris et al., 2020)
import numpy as np

In [2]:
#tensorflow is used for machine learning (Abadi et al., 2015)
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

tf.keras.utils.set_random_seed(43)  # sets seed for reproducibility, remove to get a different run - results may vary
tf.config.experimental.enable_op_determinism()

# Introduction

I have been tasked by the New York City Emergency Services Unit with investigating the relationship between weather conditions, day of week, and number of traffic collisions. Previously, I explored NOAA weather data and NYPD motor vehicle collision data in order to study this relationship.
The analysis showed a positive correlation between number of collisions and:
*   mean temperature
*   fog
*   mean dewpoint
*   maximum temperature
*   minimum temperature

It also showed a negative correlation between number of collisions and:
*   mean visibility
*   mean wind speed
*   maximum sustained wind speed
*   maximum wind gust

The best fitting linear model included:
*   mean temperature
*   mean dewpoint
*   mean visibility
*   mean wind speed
*   maximum sustained wind speed
*   maximum wind gust
*   total precipitation
*   day of week

The aim of this report is be able to predict how many traffic collisions are likely to occur on a particular day. In order to do this, I will build both linear regression models and deep neural network (DNN) models to try to predict the number of collisions on a particular day based on available data. The goal is to develop a prediction model to assist the New York City Emergency Services Unit with planning and optimization, ensuring that resources are directed to where they are most needed.

# Methodology
I used the previously gathered, collated, and cleaned data from the NOAA Global Surface Summary of the Day (noaa_gsod) (NOAA National Centers of Environmental Information, 1999) and NYPD Motor Vehicle Collisions (new_york_mv_collisions) (New York Police Department, 2017) datasets. Github link: https://raw.githubusercontent.com/23013093uhi/23013093_DataAnalytics/main/collated_collision_data_clean.csv


Both linear regession and deep neural network (DNN) models were trained and tested on the data. I created a dataframe from the .csv file and shuffled the data in order to prevent bias. For each set of variables tested, I created an input dataframe from the shuffled data and removed rows with NA values, as the regression cannot handle them. I created a training set consisting of 80% of the data and a test set of the remaining 20%. I also created a validation set (5%). I used the training set to train the model and the test set to test it. The mean absolute error was used to evaluate and compare models due to the amount of outliers. I also used the validation set to compare actual and predicted values to further assess the models.

##Linear Modelling
I created a linear regression model using all of the variables as well as one using only the variables previously determined to be significant when fitting a linear model:
* mean temperature
* mean dewpoint
* mean visibility
* mean wind speed
* maximum sustained wind speed
* maximum wind gust
* total precipitation
* day of week

I also tried removing the significant variables one by one and creating linear regression models to see if mean absolute error could be further reduced.

##DNN Modelling
I created a DNN model using all of the variables. Next, I tried removing the variables one by one and creating DNN models to see if mean absolute error could be reduced. I then tried taking all of the variables that when removed individually reduced mean absolute error and removed them all from the model. I also repeated this process while increasing the number of neurons in the dense layer from 48 to 100 to see if mean absolute error could be further reduced. I also tried increasing the number of layers in the DNN from 2 to 3. Finally, I tried creating a DNN model with only the significant variables from the linear model.


In [3]:
# create dataframe from previously created csv file of clean collated weather collison data
df = pd.read_csv('https://raw.githubusercontent.com/23013093uhi/23013093_DataAnalytics/main/collated_collision_data_clean.csv', index_col=0, )

#print the first 6 lines to check the data
print(df[:6])

   day  year  mo  da collision_date  temp  dewp     slp  visib  wdsp  mxpsd  \
1    7  2018   9  23     2018-09-23  59.8  50.2  1023.4   10.0   3.0    5.1   
2    6  2021  11   6     2021-11-06  43.1  29.7  1028.9   10.0   3.1    6.0   
3    5  2019   3   1     2019-03-01  26.7  18.2  1026.9   10.0   4.0    7.0   
4    4  2018  12  20     2018-12-20  38.6  34.4  1020.2    9.6   5.0    7.0   
5    1  2019   7   8     2019-07-08  64.9  56.7  1014.7   10.0   4.2    7.0   
6    1  2024   7   8     2024-07-08  72.0  71.5  1016.6    4.0   4.2    7.0   

   gust   max   min  prcp  sndp  fog  NUM_COLLISIONS  
1   NaN  78.1  53.1   0.0   NaN    0             475  
2   NaN  55.0  32.0   0.0   NaN    0             335  
3   NaN  35.1  16.0   0.0   NaN    0             711  
4   NaN  48.0  21.0   0.0   NaN    0             806  
5   NaN  75.0  54.0   0.0   NaN    0             592  
6   NaN  78.1  68.0   0.0   NaN    1             238  


In [4]:
# shuffle the data to randomise it in order to prevent bias

#take a sample of the entire dataset (fraction = 1) thereby shuffling it. Set random_state = 0 for reproducibility
df_shuffle = df.sample(frac=1, random_state=0)

#print first 6 lines to check data
print(df_shuffle[:6])

      day  year  mo  da collision_date  temp  dewp     slp  visib  wdsp  \
1785    3  2019   6  12     2019-06-12  62.9  51.9  1019.8   10.0   8.2   
2807    7  2020   6   7     2020-06-07  62.2  56.2  1007.9    8.8  12.3   
2476    5  2016   7   1     2016-07-01  66.9  63.6  1017.1    7.8   8.7   
2288    3  2019   6   5     2019-06-05  59.8  55.0  1014.3    9.6  12.3   
1697    2  2018   6  12     2018-06-12  55.3  48.6  1022.7   10.0   9.6   
640     1  2017  12  18     2017-12-18  32.7  23.9  1021.3    9.7   4.6   

      mxpsd  gust   max   min  prcp  sndp  fog  NUM_COLLISIONS  
1785   15.0   NaN  73.9  57.0  0.57   NaN    0             686  
2807   18.1  22.9  66.0  57.9  0.01   NaN    1             206  
2476   17.1   NaN  77.0  61.0  0.00   NaN    1             752  
2288   15.9  21.0  66.0  51.1  0.00   NaN    0             655  
1697   15.0   NaN  64.0  46.0  0.00   NaN    0             720  
640    11.1   NaN  43.0  24.1  0.00   NaN    0             706  


In [5]:
# create a scale - I used 1200 based on the maximum value of NUM_COLLISIONS
SCALE_NUM_COLLISIONS = 1200

# Results
In this run, the lowest mean absolute error was achieved by using the linear model with all significant variables:
*   mean temperature
*   mean dewpoint
*   mean visibility
*   mean wind speed
*   maximum sustained wind speed
*   maximum wind gust
*   total precipitation
*   day of week

The predicted number of collisions also appears reasonable compared to the actual values. I have set the seed so that the result shown will match the write-up. The results can vary if the seed is not set, but every time I ran the models the lowest mean absolute error was achieved by using one of the linear models with some or all of these significant variables. The DNN model with the lowest mean absolute error was achieved by removing all of the variables whose individual removal lowered the mean absolute error. Increasing either the number of layers or the number of neurons in the DNN model did not further decrease mean absolute error. The mean absolute error from the best DNN model was still higher than that obtained from the linear model with all significant variables.


### All Variables

In [6]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers = ["day", "temp", "dewp", "slp", "visib", "wdsp", "mxpsd", "gust", "max", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input = pd.concat(df_input_data, axis=1, keys=df_input_headers)

#drop the rows with NaN values (regression can't handle this)
df_input = df_input.dropna(axis=0)

#print first 6 rows to check the data
print(df_input[:6])

      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  \
2807    7  62.2  56.2  1007.9    8.8  12.3   18.1  22.9  66.0  57.9  0.01   
2288    3  59.8  55.0  1014.3    9.6  12.3   15.9  21.0  66.0  51.1  0.00   
3995    2  48.3  45.7  1010.6    3.4  17.3   25.1  35.0  54.0  41.0  0.57   
2333    1  37.8  31.6  1018.6    9.9  13.3   17.1  21.0  46.0  25.0  0.00   
4127    4  37.7  24.6  1017.9    8.2  19.9   27.0  42.9  51.1  30.0  0.24   
2973    1  68.3  64.1  1020.1   10.0  11.2   19.0  22.0  75.0  50.0  0.00   

      fog  NUM_COLLISIONS  
2807    1             206  
2288    0             655  
3995    1             608  
2333    0             238  
4127    0             252  
2973    0             521  


In [7]:
# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set = df_input.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set = df_input.drop(training_set.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set = df_input.sample(frac=0.05, random_state=0)

In [8]:
# create a copy of the datasets
training_features = training_set.copy()
valid_features = valid_set.copy()
test_features = test_set.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels = training_features.pop('NUM_COLLISIONS')
test_labels = test_features.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features.pop('NUM_COLLISIONS')

,NUM_COLLISIONS
1504,181
2408,519
3567,472
1720,279
2977,328
...,...
4357,233
3720,432
883,480
1273,474


In [9]:
# divide by the scale factor (1200) to scale the data
training_labels = training_labels/SCALE_NUM_COLLISIONS
test_labels = test_labels/SCALE_NUM_COLLISIONS

In [10]:
# use the training set training_features for the normalisation layer in order to try and fit to the output
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features))

In [11]:
# create model and add normaliser
model_0 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

In [12]:
# create a sequential model, with designated optimiser and loss parameter mean absolute error
model_0.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [13]:
# use training features and labels to fit the model. Run 100 times and include 20% validation split.

%%time
history = model_0.fit(
    training_features,
    training_labels,
    epochs=100,
    verbose=0,
    validation_split = 0.2)

CPU times: user 14.1 s, sys: 716 ms, total: 14.8 s
Wall time: 17.9 s


In [14]:
# Evaluate model using the test features and labels. Obtain mean absolute error
mean_absolute_error_model_0 = model_0.evaluate(
    test_features,
    test_labels, verbose=0)

In [15]:
# Print the mean absolute error of the model
print(mean_absolute_error_model_0)

0.1782083511352539


In [16]:
# test the model on the validation set and compare predicted with actual
linear_day_predictions_0 = model_0.predict(valid_features)*1200 #scale factor

#create a copy of the validation set
valid_set0 = valid_set.copy()

#add a column for the predicted number of collisions using our model
valid_set0["NUM_COLLISIONS_predicted"] = linear_day_predictions_0.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set0)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  \
1504    6  41.2  37.6  1017.1    9.1   7.5   14.0  20.0  48.9  37.9  0.10   
2408    7  69.4  59.9  1014.4    9.5  11.8   17.1  22.9  75.0  64.0  0.00   
3567    7  39.4  34.8  1021.0    8.3  12.4   21.0  29.9  45.0  33.1  0.55   
1720    4  61.0  61.0  1005.3    4.2   8.6   15.0  22.0  66.9  54.0  0.03   
2977    6  61.4  58.0  1019.9    6.5  11.1   19.0  28.0  66.9  52.0  0.08   
...   ...   ...   ...     ...    ...   ...    ...   ...   ...   ...   ...   
4357    4  38.9  36.3  1010.4    4.0  22.0   34.0  45.1  46.9  26.1  0.20   
3720    7  56.5  48.2  1026.6   10.0  15.5   22.0  25.1  63.0  51.1  0.00   
883     7  63.8  50.3  1015.4    9.0   6.4   12.0  15.9  72.0  55.4  0.22   
1273    6  75.7  65.2  1023.6    7.3   3.2   13.0  18.1  84.0  62.6  0.01   
3173    4  30.9  10.8  1015.8   10.0  14.5   20.0  27.0  37.0  25.0  0.00   

      fog  NUM_COLLISIONS NUM_COLLISI

### All Significant Variables

In [17]:
# create an input dataframe from the shuffled dataframe
#get the data
df_input_data1 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers1 = ["day", "temp", "dewp", "visib", "wdsp", "mxpsd", "gust", "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input1 = pd.concat(df_input_data1, axis=1, keys=df_input_headers1)

#drop the rows with NaN values (regression can't handle this)
df_input1 = df_input1.dropna(axis=0)

#print first 6 rows to check the data
print(df_input1[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set1 = df_input1.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set1 = df_input1.drop(training_set1.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set1 = df_input1.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features1 = training_set1.copy()
valid_features1 = valid_set1.copy()
test_features1 = test_set1.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels1 = training_features1.pop('NUM_COLLISIONS')
test_labels1 = test_features1.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features1.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels1 = training_labels1/SCALE_NUM_COLLISIONS
test_labels1 = test_labels1/SCALE_NUM_COLLISIONS

# use the training set training_features for the normalisation layer in order to try and fit to the output
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features1))

# create model and add normaliser
model_1 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

# create a sequential model, with designated optimiser and loss parameter mean absolute error
model_1.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

      day  temp  dewp  visib  wdsp  mxpsd  gust  NUM_COLLISIONS
2807    7  62.2  56.2    8.8  12.3   18.1  22.9             206
2288    3  59.8  55.0    9.6  12.3   15.9  21.0             655
3995    2  48.3  45.7    3.4  17.3   25.1  35.0             608
2333    1  37.8  31.6    9.9  13.3   17.1  21.0             238
4127    4  37.7  24.6    8.2  19.9   27.0  42.9             252
2973    1  68.3  64.1   10.0  11.2   19.0  22.0             521


In [18]:
# use training features and labels to fit the model. Run 100 times and include 20% validation split.

%%time
history = model_1.fit(
    training_features1,
    training_labels1,
    epochs=100,
    verbose=0,
    validation_split = 0.2)


CPU times: user 13.6 s, sys: 680 ms, total: 14.2 s
Wall time: 16.3 s


In [19]:
# Evaluate model using the test features and labels. Obtain mean absolute error
mean_absolute_error_model_1 = model_1.evaluate(
    test_features1,
    test_labels1, verbose=0)

# Print the mean absolute error of the model
print(mean_absolute_error_model_1)

# test the model on the validation set and compare predicted with actual
linear_day_predictions_1 = model_1.predict(valid_features1)*1200 # scale factor

#create a copy of the validation set
valid_set1a = valid_set1.copy()

#add a column for the predicted number of collisions using our model
valid_set1a["NUM_COLLISIONS_predicted"] = linear_day_predictions_1.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set1a)

0.1279795616865158
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
      day  temp  dewp  visib  wdsp  mxpsd  gust  NUM_COLLISIONS  \
3271    1  62.3  58.2    8.8  13.8   20.0  27.0             641   
3709    7  48.8  39.8    9.6  11.9   22.0  28.9             241   
3135    7  49.6  47.9    6.0  15.3   19.0  28.0             256   
4263    4  43.7  39.1    5.3  16.4   28.9  36.9             266   
1746    4  42.7  35.1    9.8   6.4   15.0  22.9             635   
...   ...   ...   ...    ...   ...    ...   ...             ...   
2771    2  63.3  59.7    8.7  14.1   18.1  22.9             318   
2355    2  37.1  22.6   10.0  12.2   17.1  24.1             282   
2506    5  53.6  44.3   10.0  12.1   17.1  22.9             777   
3595    7  37.8  34.1    9.1  13.4   22.0  28.0             541   
2922    3  42.1  38.5    5.0  12.3   19.0  25.1             565   

     NUM_COLLISIONS_predicted  
3271      [538.3699340820312]  
3709     [504.58038330078125]  
3135     [404.02838134765625]  
4263      

### Remove Mean Temperature

In [20]:
# create an input dataframe from the shuffled dataframe
#get the data
df_input_data2 = [df_shuffle["day"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers2 = ["day", "dewp", "visib", "wdsp", "mxpsd", "gust", "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input2 = pd.concat(df_input_data2, axis=1, keys=df_input_headers2)

#drop the rows with NaN values (regression can't handle this)
df_input2 = df_input2.dropna(axis=0)

#print first 6 rows to check the data
print(df_input2[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set
#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set2 = df_input2.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set2 = df_input2.drop(training_set2.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set2 = df_input2.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features2 = training_set2.copy()
valid_features2 = valid_set2.copy()
test_features2 = test_set2.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels2 = training_features2.pop('NUM_COLLISIONS')
test_labels2 = test_features2.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features2.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels2 = training_labels2/SCALE_NUM_COLLISIONS
test_labels2 = test_labels2/SCALE_NUM_COLLISIONS

# use the training set training_features for the normalisation layer in order to try and fit to the output
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features2))

# create model and add normaliser
model_2 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

# create a sequential model, with designated optimiser and loss parameter mean absolute error
model_2.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

      day  dewp  visib  wdsp  mxpsd  gust  NUM_COLLISIONS
2807    7  56.2    8.8  12.3   18.1  22.9             206
2288    3  55.0    9.6  12.3   15.9  21.0             655
3995    2  45.7    3.4  17.3   25.1  35.0             608
2333    1  31.6    9.9  13.3   17.1  21.0             238
4127    4  24.6    8.2  19.9   27.0  42.9             252
2973    1  64.1   10.0  11.2   19.0  22.0             521


In [21]:
# use training features and labels to fit the model. Run 100 times and include 20% validation split.

%%time
history = model_2.fit(
    training_features2,
    training_labels2,
    epochs=100,
    verbose=0,
    validation_split = 0.2)

CPU times: user 13.3 s, sys: 672 ms, total: 14 s
Wall time: 14.8 s


In [22]:
# Evaluate model using the test features and labels. Obtain mean absolute error
mean_absolute_error_model_2 = model_2.evaluate(
    test_features2,
    test_labels2, verbose=0)

# Print the mean absolute error of the model
print(mean_absolute_error_model_2)

# test the model on the validation set and compare predicted with actual
linear_day_predictions_2 = model_2.predict(valid_features2)*1200 # essentially 600000 in this instance would give back realistic numbers based on the TAXI_TRIPS data

#create a copy of the validation set
valid_set2a = valid_set2.copy()

#add a column for the predicted number of collisions using our model
valid_set2a["NUM_COLLISIONS_predicted"] = linear_day_predictions_2.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set2a)

0.143507719039917
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
      day  dewp  visib  wdsp  mxpsd  gust  NUM_COLLISIONS  \
3271    1  58.2    8.8  13.8   20.0  27.0             641   
3709    7  39.8    9.6  11.9   22.0  28.9             241   
3135    7  47.9    6.0  15.3   19.0  28.0             256   
4263    4  39.1    5.3  16.4   28.9  36.9             266   
1746    4  35.1    9.8   6.4   15.0  22.9             635   
...   ...   ...    ...   ...    ...   ...             ...   
2771    2  59.7    8.7  14.1   18.1  22.9             318   
2355    2  22.6   10.0  12.2   17.1  24.1             282   
2506    5  44.3   10.0  12.1   17.1  22.9             777   
3595    7  34.1    9.1  13.4   22.0  28.0             541   
2922    3  38.5    5.0  12.3   19.0  25.1             565   

     NUM_COLLISIONS_predicted  
3271      [459.8818664550781]  
3709     [500.35137939453125]  
3135     [318.15301513671875]  
4263      [367.7852478027344]  
1746     [506.82220458984375]  
...                       ...  
2771 

### Remove Mean Dewpoint

In [23]:
# create an input dataframe from the shuffled dataframe
#get the data
df_input_data3 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers3 = ["day", "temp", "visib", "wdsp", "mxpsd", "gust", "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input3 = pd.concat(df_input_data3, axis=1, keys=df_input_headers3)

#drop the rows with NaN values (regression can't handle this)
df_input3 = df_input3.dropna(axis=0)

#print first 6 rows to check the data
print(df_input3[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set3 = df_input3.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set3 = df_input3.drop(training_set3.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set3 = df_input3.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features3 = training_set2.copy()
valid_features3 = valid_set3.copy()
test_features3 = test_set3.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels3 = training_features3.pop('NUM_COLLISIONS')
test_labels3 = test_features3.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features3.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels3 = training_labels3/SCALE_NUM_COLLISIONS
test_labels3 = test_labels3/SCALE_NUM_COLLISIONS

# use the training set training_features for the normalisation layer in order to try and fit to the output
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features3))

# create model and add normaliser
model_3 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

# create a sequential model, with designated optimiser and loss parameter mean absolute error
model_3.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')


      day  temp  visib  wdsp  mxpsd  gust  NUM_COLLISIONS
2807    7  62.2    8.8  12.3   18.1  22.9             206
2288    3  59.8    9.6  12.3   15.9  21.0             655
3995    2  48.3    3.4  17.3   25.1  35.0             608
2333    1  37.8    9.9  13.3   17.1  21.0             238
4127    4  37.7    8.2  19.9   27.0  42.9             252
2973    1  68.3   10.0  11.2   19.0  22.0             521


In [24]:
# use training features and labels to fit the model. Run 100 times and include 20% validation split.

%%time
history = model_3.fit(
    training_features3,
    training_labels3,
    epochs=100,
    verbose=0,
    validation_split = 0.2)

CPU times: user 13.3 s, sys: 648 ms, total: 13.9 s
Wall time: 14.4 s


In [25]:
# Evaluate model using the test features and labels. Obtain mean absolute error
mean_absolute_error_model_3 = model_3.evaluate(
    test_features3,
    test_labels3, verbose=0)

# Print the mean absolute error of the model
print(mean_absolute_error_model_3)

# test the model on the validation set and compare predicted with actual
linear_day_predictions_3 = model_3.predict(valid_features3)*1200 # scale factor

#create a copy of the validation set
valid_set3a = valid_set3.copy()

#add a column for the predicted number of collisions using our model
valid_set3a["NUM_COLLISIONS_predicted"] = linear_day_predictions_3.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set3a)

0.13664111495018005
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
      day  temp  visib  wdsp  mxpsd  gust  NUM_COLLISIONS  \
3271    1  62.3    8.8  13.8   20.0  27.0             641   
3709    7  48.8    9.6  11.9   22.0  28.9             241   
2115    2  50.8    9.9  10.6   15.9  18.1             312   
3298    2  43.1   10.0  17.5   20.0  28.0             340   
3679    5  28.0   10.0  13.1   22.0  29.9             702   
...   ...   ...    ...   ...    ...   ...             ...   
2789    5  44.8    2.9   8.8   18.1  26.0             582   
3775    6  42.1   10.0  16.7   22.9  32.1             624   
3512    6  43.8    5.0  11.8   21.0  31.1             455   
3595    7  37.8    9.1  13.4   22.0  28.0             541   
2686    3  53.9    9.0   9.2   18.1  26.0             641   

     NUM_COLLISIONS_predicted  
3271      [507.5648498535156]  
3709       [517.657958984375]  
2115      [527.4697875976562]  
3298      [516.9403076171875]  
3679      [560.1661987304688]  
...              

### Remove Mean Visibility

In [26]:
# create an input dataframe from the shuffled dataframe
#get the data
df_input_data4 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers4 = ["day", "temp", "dewp", "wdsp", "mxpsd", "gust", "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input4 = pd.concat(df_input_data4, axis=1, keys=df_input_headers4)

#drop the rows with NaN values (regression can't handle this)
df_input4 = df_input4.dropna(axis=0)

#print first 6 rows to check the data
print(df_input4[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set4 = df_input4.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set4 = df_input4.drop(training_set4.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set4 = df_input4.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features4 = training_set4.copy()
valid_features4 = valid_set4.copy()
test_features4 = test_set4.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels4 = training_features4.pop('NUM_COLLISIONS')
test_labels4 = test_features4.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features4.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels4 = training_labels4/SCALE_NUM_COLLISIONS
test_labels4 = test_labels4/SCALE_NUM_COLLISIONS

# use the training set training_features for the normalisation layer in order to try and fit to the output
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features4))

# create model and add normaliser
model_4 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

# create a sequential model, with designated optimiser and loss parameter mean absolute error
model_4.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')


      day  temp  dewp  wdsp  mxpsd  gust  NUM_COLLISIONS
2807    7  62.2  56.2  12.3   18.1  22.9             206
2288    3  59.8  55.0  12.3   15.9  21.0             655
3995    2  48.3  45.7  17.3   25.1  35.0             608
2333    1  37.8  31.6  13.3   17.1  21.0             238
4127    4  37.7  24.6  19.9   27.0  42.9             252
2973    1  68.3  64.1  11.2   19.0  22.0             521


In [27]:
# use training features and labels to fit the model. Run 100 times and include 20% validation split.

%%time
history = model_4.fit(
    training_features4,
    training_labels4,
    epochs=100,
    verbose=0,
    validation_split = 0.2)

CPU times: user 13.2 s, sys: 640 ms, total: 13.8 s
Wall time: 14.7 s


In [28]:
# Evaluate model using the test features and labels. Obtain mean absolute error
mean_absolute_error_model_4 = model_4.evaluate(
    test_features4,
    test_labels4, verbose=0)

# Print the mean absolute error of the model
print(mean_absolute_error_model_4)

# test the model on the validation set and compare predicted with actual
linear_day_predictions_4 = model_4.predict(valid_features4)*1200 # essentially 600000 in this instance would give back realistic numbers based on the TAXI_TRIPS data

#create a copy of the validation set
valid_set4a = valid_set4.copy()

#add a column for the predicted number of collisions using our model
valid_set4a["NUM_COLLISIONS_predicted"] = linear_day_predictions_4.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set4a)

0.13224492967128754
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
      day  temp  dewp  wdsp  mxpsd  gust  NUM_COLLISIONS  \
3271    1  62.3  58.2  13.8   20.0  27.0             641   
3709    7  48.8  39.8  11.9   22.0  28.9             241   
3135    7  49.6  47.9  15.3   19.0  28.0             256   
4263    4  43.7  39.1  16.4   28.9  36.9             266   
1746    4  42.7  35.1   6.4   15.0  22.9             635   
...   ...   ...   ...   ...    ...   ...             ...   
2771    2  63.3  59.7  14.1   18.1  22.9             318   
2355    2  37.1  22.6  12.2   17.1  24.1             282   
2506    5  53.6  44.3  12.1   17.1  22.9             777   
3595    7  37.8  34.1  13.4   22.0  28.0             541   
2922    3  42.1  38.5  12.3   19.0  25.1             565   

     NUM_COLLISIONS_predicted  
3271      [594.3936157226562]  
3709      [500.6744079589844]  
3135     [423.32659912109375]  
4263     [494.33111572265625]  
1746      [619.2216186523438]  
...                       ...

### Remove Mean Wind Speed

In [29]:
# create an input dataframe from the shuffled dataframe
#get the data
df_input_data5 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["mxpsd"], df_shuffle["gust"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers5 = ["day", "temp", "dewp", "visib", "mxpsd", "gust", "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input5 = pd.concat(df_input_data5, axis=1, keys=df_input_headers5)

#drop the rows with NaN values (regression can't handle this)
df_input5 = df_input5.dropna(axis=0)

#print first 6 rows to check the data
print(df_input5[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set5 = df_input5.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set5 = df_input5.drop(training_set5.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set5 = df_input5.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features5 = training_set5.copy()
valid_features5 = valid_set5.copy()
test_features5 = test_set5.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels5 = training_features5.pop('NUM_COLLISIONS')
test_labels5 = test_features5.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features5.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels5 = training_labels5/SCALE_NUM_COLLISIONS
test_labels5 = test_labels5/SCALE_NUM_COLLISIONS

# use the training set training_features for the normalisation layer in order to try and fit to the output
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features5))

# create model and add normaliser
model_5 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

# create a sequential model, with designated optimiser and loss parameter mean absolute error
model_5.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')


      day  temp  dewp  visib  mxpsd  gust  NUM_COLLISIONS
2807    7  62.2  56.2    8.8   18.1  22.9             206
2288    3  59.8  55.0    9.6   15.9  21.0             655
3995    2  48.3  45.7    3.4   25.1  35.0             608
2333    1  37.8  31.6    9.9   17.1  21.0             238
4127    4  37.7  24.6    8.2   27.0  42.9             252
2973    1  68.3  64.1   10.0   19.0  22.0             521


In [30]:
# use training features and labels to fit the model. Run 100 times and include 20% validation split.

%%time
history = model_5.fit(
    training_features5,
    training_labels5,
    epochs=100,
    verbose=0,
    validation_split = 0.2)

CPU times: user 13.6 s, sys: 681 ms, total: 14.3 s
Wall time: 16.1 s


In [31]:
# Evaluate model using the test features and labels. Obtain mean absolute error
mean_absolute_error_model_5 = model_5.evaluate(
    test_features5,
    test_labels5, verbose=0)

# Print the mean absolute error of the model
print(mean_absolute_error_model_5)

# test the model on the validation set and compare predicted with actual
linear_day_predictions_5 = model_5.predict(valid_features5)*1200 #scale factor

#create a copy of the validation set
valid_set5a = valid_set5.copy()

#add a column for the predicted number of collisions using our model
valid_set5a["NUM_COLLISIONS_predicted"] = linear_day_predictions_5.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set5a)

0.13423854112625122
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
      day  temp  dewp  visib  mxpsd  gust  NUM_COLLISIONS  \
3271    1  62.3  58.2    8.8   20.0  27.0             641   
3709    7  48.8  39.8    9.6   22.0  28.9             241   
3135    7  49.6  47.9    6.0   19.0  28.0             256   
4263    4  43.7  39.1    5.3   28.9  36.9             266   
1746    4  42.7  35.1    9.8   15.0  22.9             635   
...   ...   ...   ...    ...    ...   ...             ...   
2771    2  63.3  59.7    8.7   18.1  22.9             318   
2355    2  37.1  22.6   10.0   17.1  24.1             282   
2506    5  53.6  44.3   10.0   17.1  22.9             777   
3595    7  37.8  34.1    9.1   22.0  28.0             541   
2922    3  42.1  38.5    5.0   19.0  25.1             565   

     NUM_COLLISIONS_predicted  
3271      [663.5039672851562]  
3709      [451.9273376464844]  
3135     [476.23614501953125]  
4263      [551.1253051757812]  
1746      [508.0480041503906]  
...              

### Remove Maximum Sustained Wind Speed

In [32]:
# create an input dataframe from the shuffled dataframe
#get the data
df_input_data6 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["gust"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers6 = ["day", "temp", "dewp", "visib", "wdsp", "gust", "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input6 = pd.concat(df_input_data6, axis=1, keys=df_input_headers6)

#drop the rows with NaN values (regression can't handle this)
df_input6 = df_input6.dropna(axis=0)

#print first 6 rows to check the data
print(df_input6[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set6 = df_input6.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set6 = df_input6.drop(training_set6.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set6 = df_input6.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features6 = training_set6.copy()
valid_features6 = valid_set6.copy()
test_features6 = test_set6.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels6 = training_features6.pop('NUM_COLLISIONS')
test_labels6 = test_features6.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features6.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels6 = training_labels6/SCALE_NUM_COLLISIONS
test_labels6 = test_labels6/SCALE_NUM_COLLISIONS

# use the training set training_features for the normalisation layer in order to try and fit to the output
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features6))

# create model and add normaliser
model_6 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

# create a sequential model, with designated optimiser and loss parameter mean absolute error
model_6.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')


      day  temp  dewp  visib  wdsp  gust  NUM_COLLISIONS
2807    7  62.2  56.2    8.8  12.3  22.9             206
2288    3  59.8  55.0    9.6  12.3  21.0             655
3995    2  48.3  45.7    3.4  17.3  35.0             608
2333    1  37.8  31.6    9.9  13.3  21.0             238
4127    4  37.7  24.6    8.2  19.9  42.9             252
2973    1  68.3  64.1   10.0  11.2  22.0             521


In [33]:
# use training features and labels to fit the model. Run 100 times and include 20% validation split.

%%time
history = model_6.fit(
    training_features6,
    training_labels6,
    epochs=100,
    verbose=0,
    validation_split = 0.2)



CPU times: user 13.4 s, sys: 679 ms, total: 14 s
Wall time: 15.6 s


In [34]:
# Evaluate model using the test features and labels. Obtain mean absolute error
mean_absolute_error_model_6 = model_6.evaluate(
    test_features6,
    test_labels6, verbose=0)

# Print the mean absolute error of the model
print(mean_absolute_error_model_6)

# test the model on the validation set and compare predicted with actual
linear_day_predictions_6 = model_6.predict(valid_features6)*1200 # essentially 600000 in this instance would give back realistic numbers based on the TAXI_TRIPS data

#create a copy of the validation set
valid_set6a = valid_set6.copy()

#add a column for the predicted number of collisions using our model
valid_set6a["NUM_COLLISIONS_predicted"] = linear_day_predictions_6.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set6a)


0.1345731019973755
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
      day  temp  dewp  visib  wdsp  gust  NUM_COLLISIONS  \
3271    1  62.3  58.2    8.8  13.8  27.0             641   
3709    7  48.8  39.8    9.6  11.9  28.9             241   
3135    7  49.6  47.9    6.0  15.3  28.0             256   
4263    4  43.7  39.1    5.3  16.4  36.9             266   
1746    4  42.7  35.1    9.8   6.4  22.9             635   
...   ...   ...   ...    ...   ...   ...             ...   
2771    2  63.3  59.7    8.7  14.1  22.9             318   
2355    2  37.1  22.6   10.0  12.2  24.1             282   
2506    5  53.6  44.3   10.0  12.1  22.9             777   
3595    7  37.8  34.1    9.1  13.4  28.0             541   
2922    3  42.1  38.5    5.0  12.3  25.1             565   

     NUM_COLLISIONS_predicted  
3271      [692.6053466796875]  
3709      [440.5789489746094]  
3135      [401.3814697265625]  
4263     [481.65582275390625]  
1746      [495.0378723144531]  
...                       ... 

### Remove Maximum Wind Gust

In [35]:
# create an input dataframe from the shuffled dataframe
#get the data
df_input_data7 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers7 = ["day", "temp", "dewp", "visib", "wdsp", "mxpsd", "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input7 = pd.concat(df_input_data7, axis=1, keys=df_input_headers7)

#drop the rows with NaN values (regression can't handle this)
df_input7 = df_input7.dropna(axis=0)

#print first 6 rows to check the data
print(df_input7[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set7 = df_input7.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set7 = df_input7.drop(training_set7.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set7 = df_input7.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features7 = training_set7.copy()
valid_features7 = valid_set7.copy()
test_features7 = test_set7.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels7 = training_features7.pop('NUM_COLLISIONS')
test_labels7 = test_features7.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features7.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels7 = training_labels7/SCALE_NUM_COLLISIONS
test_labels7 = test_labels7/SCALE_NUM_COLLISIONS

# use the training set training_features for the normalisation layer in order to try and fit to the output
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features7))

# create model and add normaliser
model_7 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

# create a sequential model, with designated optimiser and loss parameter mean absolute error
model_7.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')


      day  temp  dewp  visib  wdsp  mxpsd  NUM_COLLISIONS
1785    3  62.9  51.9   10.0   8.2   15.0             686
2807    7  62.2  56.2    8.8  12.3   18.1             206
2476    5  66.9  63.6    7.8   8.7   17.1             752
2288    3  59.8  55.0    9.6  12.3   15.9             655
1697    2  55.3  48.6   10.0   9.6   15.0             720
640     1  32.7  23.9    9.7   4.6   11.1             706


In [36]:
# use training features and labels to fit the model. Run 100 times and include 20% validation split.

%%time
history = model_7.fit(
    training_features7,
    training_labels7,
    epochs=100,
    verbose=0,
    validation_split = 0.2)


CPU times: user 18.7 s, sys: 1 s, total: 19.7 s
Wall time: 22.6 s


In [37]:
# Evaluate model using the test features and labels. Obtain mean absolute error
mean_absolute_error_model_7 = model_7.evaluate(
    test_features7,
    test_labels7, verbose=0)

# Print the mean absolute error of the model
print(mean_absolute_error_model_7)

# test the model on the validation set and compare predicted with actual
linear_day_predictions_7 = model_7.predict(valid_features7)*1200 # essentially 600000 in this instance would give back realistic numbers based on the TAXI_TRIPS data

#create a copy of the validation set
valid_set7a = valid_set7.copy()

#add a column for the predicted number of collisions using our model
valid_set7a["NUM_COLLISIONS_predicted"] = linear_day_predictions_7.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set7a)


0.1355268508195877
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
      day  temp  dewp  visib  wdsp  mxpsd  NUM_COLLISIONS  \
3727    4  24.9   4.8   10.0  19.2   22.9             412   
3834    7  51.9  51.2    2.1  15.2   22.9             505   
2859    6  44.9  36.6    9.4  12.2   18.1             549   
3822    2  32.6  25.6    8.4  16.3   22.9             613   
3578    2  60.5  55.1    8.7  14.4   21.0             715   
...   ...   ...   ...    ...   ...    ...             ...   
3537    4  46.6  41.2   10.0  12.7   21.0             197   
2914    4  29.9  11.8   10.0  12.2   19.0             625   
2765    5  61.7  58.6    4.0  14.4   18.1             708   
2248    7  71.5  70.6    7.0  11.9   15.9             284   
2010    3  41.1  38.4    6.5   6.0   15.9             583   

     NUM_COLLISIONS_predicted  
3727     [410.91107177734375]  
3834     [240.20681762695312]  
2859      [457.9117736816406]  
3822       [398.500732421875]  
3578     [454.98089599609375]  
...               

### Remove Day of Week

In [38]:
# create an input dataframe from the shuffled dataframe
#get the data
df_input_data8 = [df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers8 = ["temp", "dewp", "visib", "wdsp", "mxpsd", "gust", "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input8 = pd.concat(df_input_data8, axis=1, keys=df_input_headers8)

#drop the rows with NaN values (regression can't handle this)
df_input8 = df_input8.dropna(axis=0)

#print first 6 rows to check the data
print(df_input8[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set8 = df_input8.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set8 = df_input8.drop(training_set8.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set8 = df_input8.sample(frac=0.05, random_state=0)

# create a copy of the datasets.
training_features8 = training_set8.copy()
valid_features8 = valid_set8.copy()
test_features8 = test_set8.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels8 = training_features8.pop('NUM_COLLISIONS')
test_labels8 = test_features8.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features8.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels8 = training_labels8/SCALE_NUM_COLLISIONS
test_labels8 = test_labels8/SCALE_NUM_COLLISIONS

# use the training set training_features for the normalisation layer in order to try and fit to the output
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features8))

# create model and add normaliser
model_8 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

# create a sequential model, with designated optimiser and loss parameter mean absolute error
model_8.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')



      temp  dewp  visib  wdsp  mxpsd  gust  NUM_COLLISIONS
2807  62.2  56.2    8.8  12.3   18.1  22.9             206
2288  59.8  55.0    9.6  12.3   15.9  21.0             655
3995  48.3  45.7    3.4  17.3   25.1  35.0             608
2333  37.8  31.6    9.9  13.3   17.1  21.0             238
4127  37.7  24.6    8.2  19.9   27.0  42.9             252
2973  68.3  64.1   10.0  11.2   19.0  22.0             521


In [39]:
# use training features and labels to fit the model. Run 100 times and include 20% validation split.

%%time
history = model_8.fit(
    training_features8,
    training_labels8,
    epochs=100,
    verbose=0,
    validation_split = 0.2)



CPU times: user 13.4 s, sys: 692 ms, total: 14.1 s
Wall time: 14.8 s


In [40]:
# Evaluate model using the test features and labels. Obtain mean absolute error
mean_absolute_error_model_8 = model_8.evaluate(
    test_features8,
    test_labels8, verbose=0)

# Print the mean absolute error of the model
print(mean_absolute_error_model_8)

# test the model on the validation set and compare predicted with actual
linear_day_predictions_8 = model_8.predict(valid_features8)*1200 # essentially 600000 in this instance would give back realistic numbers based on the TAXI_TRIPS data

#create a copy of the validation set
valid_set8a = valid_set8.copy()

#add a column for the predicted number of collisions using our model
valid_set8a["NUM_COLLISIONS_predicted"] = linear_day_predictions_8.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set8a)


0.14146864414215088
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
      temp  dewp  visib  wdsp  mxpsd  gust  NUM_COLLISIONS  \
3271  62.3  58.2    8.8  13.8   20.0  27.0             641   
3709  48.8  39.8    9.6  11.9   22.0  28.9             241   
3135  49.6  47.9    6.0  15.3   19.0  28.0             256   
4263  43.7  39.1    5.3  16.4   28.9  36.9             266   
1746  42.7  35.1    9.8   6.4   15.0  22.9             635   
...    ...   ...    ...   ...    ...   ...             ...   
2771  63.3  59.7    8.7  14.1   18.1  22.9             318   
2355  37.1  22.6   10.0  12.2   17.1  24.1             282   
2506  53.6  44.3   10.0  12.1   17.1  22.9             777   
3595  37.8  34.1    9.1  13.4   22.0  28.0             541   
2922  42.1  38.5    5.0  12.3   19.0  25.1             565   

     NUM_COLLISIONS_predicted  
3271      [560.8511352539062]  
3709      [535.4234008789062]  
3135      [455.9004211425781]  
4263      [405.0241394042969]  
1746      [548.4365844726562]  
...  

### Compare Mean Absolute Error

In [41]:
print("Mean absolute error results")
print("All variables: " + str(mean_absolute_error_model_0))
print("Significant variables: " + str(mean_absolute_error_model_1))
print("Significant variables except mean temperature: " + str(mean_absolute_error_model_2))
print("Significant variables except mean dewpoint: " + str(mean_absolute_error_model_3))
print("Significant variables except mean visibility: " + str(mean_absolute_error_model_4))
print("Significant variables except mean wind speed: " + str(mean_absolute_error_model_5))
print("Significant variables except maximum sustained wind speed: " + str(mean_absolute_error_model_6))
print("Significant variables except maximum wind gust: " + str(mean_absolute_error_model_7))
print("Significant variables except day of week: " + str(mean_absolute_error_model_8))

Mean absolute error results
All variables: 0.1782083511352539
Significant variables: 0.1279795616865158
Significant variables except mean temperature: 0.143507719039917
Significant variables except mean dewpoint: 0.13664111495018005
Significant variables except mean visibility: 0.13224492967128754
Significant variables except mean wind speed: 0.13423854112625122
Significant variables except maximum sustained wind speed: 0.1345731019973755
Significant variables except maximum wind gust: 0.1355268508195877
Significant variables except day of week: 0.14146864414215088


##DNN Modelling
###All Variables

In [42]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data9 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers9 = ["day", "temp", "dewp", "slp", "visib", "wdsp", "mxpsd", "gust", "max", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input9 = pd.concat(df_input_data9, axis=1, keys=df_input_headers9)

#drop the rows with NaN values (regression can't handle this)
df_input9 = df_input9.dropna(axis=0)

#print first 6 rows to check the data
print(df_input9[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set9 = df_input9.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set9 = df_input9.drop(training_set9.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set9 = df_input9.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features9 = training_set9.copy()
valid_features9 = valid_set9.copy()
test_features9 = test_set9.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels9 = training_features9.pop('NUM_COLLISIONS')
test_labels9 = test_features9.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features9.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels9 = training_labels9/SCALE_NUM_COLLISIONS
test_labels9 = test_labels9/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features9))

# Normalisation layer (12 inputs), 2 layers of 48, with 1 output.
dnn_model_1 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_1.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  \
2807    7  62.2  56.2  1007.9    8.8  12.3   18.1  22.9  66.0  57.9  0.01   
2288    3  59.8  55.0  1014.3    9.6  12.3   15.9  21.0  66.0  51.1  0.00   
3995    2  48.3  45.7  1010.6    3.4  17.3   25.1  35.0  54.0  41.0  0.57   
2333    1  37.8  31.6  1018.6    9.9  13.3   17.1  21.0  46.0  25.0  0.00   
4127    4  37.7  24.6  1017.9    8.2  19.9   27.0  42.9  51.1  30.0  0.24   
2973    1  68.3  64.1  1020.1   10.0  11.2   19.0  22.0  75.0  50.0  0.00   

      fog  NUM_COLLISIONS  
2807    1             206  
2288    0             655  
3995    1             608  
2333    0             238  
4127    0             252  
2973    0             521  


In [43]:
%%time
history = dnn_model_1.fit(
    training_features9,
    training_labels9,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 16 s, sys: 818 ms, total: 16.8 s
Wall time: 16.6 s


In [44]:
# evaluate the mean absolute error for the model
dnn_model_1_results = dnn_model_1.evaluate(test_features9, test_labels9, verbose=0)
print(dnn_model_1_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_1 = dnn_model_1.predict(valid_features9)*1200 #scale factor

#create a copy of the validation set
valid_set9a = valid_set9.copy()

#add a column for the predicted number of collisions using our model
valid_set9a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_1.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set9a)



0.14019443094730377
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  \
1504    6  41.2  37.6  1017.1    9.1   7.5   14.0  20.0  48.9  37.9  0.10   
2408    7  69.4  59.9  1014.4    9.5  11.8   17.1  22.9  75.0  64.0  0.00   
3567    7  39.4  34.8  1021.0    8.3  12.4   21.0  29.9  45.0  33.1  0.55   
1720    4  61.0  61.0  1005.3    4.2   8.6   15.0  22.0  66.9  54.0  0.03   
2977    6  61.4  58.0  1019.9    6.5  11.1   19.0  28.0  66.9  52.0  0.08   
...   ...   ...   ...     ...    ...   ...    ...   ...   ...   ...   ...   
4357    4  38.9  36.3  1010.4    4.0  22.0   34.0  45.1  46.9  26.1  0.20   
3720    7  56.5  48.2  1026.6   10.0  15.5   22.0  25.1  63.0  51.1  0.00   
883     7  63.8  50.3  1015.4    9.0   6.4   12.0  15.9  72.0  55.4  0.22   
1273    6  75.7  65.2  1023.6    7.3   3.2   13.0  18.1  84.0  62.6  0.01   
3173    4  30.9  10.8  1015.8   10.0  14.5   20.0  27.0  37.0  25.0  0.00   

      fog  NUM_CO

###Remove Mean Temperature

In [45]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data10 = [df_shuffle["day"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers10 = ["day", "dewp", "slp", "visib", "wdsp", "mxpsd", "gust", "max", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input10 = pd.concat(df_input_data10, axis=1, keys=df_input_headers10)

#drop the rows with NaN values (regression can't handle this)
df_input10 = df_input10.dropna(axis=0)

#print first 6 rows to check the data
print(df_input10[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set10 = df_input10.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set10 = df_input10.drop(training_set10.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set10 = df_input10.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features10 = training_set10.copy()
valid_features10 = valid_set10.copy()
test_features10 = test_set10.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels10 = training_features10.pop('NUM_COLLISIONS')
test_labels10 = test_features10.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features10.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels10 = training_labels10/SCALE_NUM_COLLISIONS
test_labels10 = test_labels10/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features10))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_2 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_2.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  dewp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  fog  \
2807    7  56.2  1007.9    8.8  12.3   18.1  22.9  66.0  57.9  0.01    1   
2288    3  55.0  1014.3    9.6  12.3   15.9  21.0  66.0  51.1  0.00    0   
3995    2  45.7  1010.6    3.4  17.3   25.1  35.0  54.0  41.0  0.57    1   
2333    1  31.6  1018.6    9.9  13.3   17.1  21.0  46.0  25.0  0.00    0   
4127    4  24.6  1017.9    8.2  19.9   27.0  42.9  51.1  30.0  0.24    0   
2973    1  64.1  1020.1   10.0  11.2   19.0  22.0  75.0  50.0  0.00    0   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [46]:
%%time
history = dnn_model_2.fit(
    training_features10,
    training_labels10,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 16.3 s, sys: 757 ms, total: 17.1 s
Wall time: 17.4 s


In [47]:
# evaluate the mean absolute error for the model
dnn_model_2_results = dnn_model_2.evaluate(test_features10, test_labels10, verbose=0)
print(dnn_model_2_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_2 = dnn_model_2.predict(valid_features10)*1200 #scale factor

#create a copy of the validation set
valid_set10a = valid_set10.copy()

#add a column for the predicted number of collisions using our model
valid_set10a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_2.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set10a)



0.1428210735321045
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
      day  dewp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  fog  \
1504    6  37.6  1017.1    9.1   7.5   14.0  20.0  48.9  37.9  0.10    0   
2408    7  59.9  1014.4    9.5  11.8   17.1  22.9  75.0  64.0  0.00    0   
3567    7  34.8  1021.0    8.3  12.4   21.0  29.9  45.0  33.1  0.55    0   
1720    4  61.0  1005.3    4.2   8.6   15.0  22.0  66.9  54.0  0.03    1   
2977    6  58.0  1019.9    6.5  11.1   19.0  28.0  66.9  52.0  0.08    0   
...   ...   ...     ...    ...   ...    ...   ...   ...   ...   ...  ...   
4357    4  36.3  1010.4    4.0  22.0   34.0  45.1  46.9  26.1  0.20    0   
3720    7  48.2  1026.6   10.0  15.5   22.0  25.1  63.0  51.1  0.00    0   
883     7  50.3  1015.4    9.0   6.4   12.0  15.9  72.0  55.4  0.22    0   
1273    6  65.2  1023.6    7.3   3.2   13.0  18.1  84.0  62.6  0.01    1   
3173    4  10.8  1015.8   10.0  14.5   20.0  27.0  37.0  25.0  0.00    0   

      NUM_COLLISIONS NUM_COLLI

###Remove Mean Dewpoint


In [48]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data11 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers11 = ["day", "temp", "slp", "visib", "wdsp", "mxpsd", "gust", "max", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input11 = pd.concat(df_input_data11, axis=1, keys=df_input_headers11)

#drop the rows with NaN values (regression can't handle this)
df_input11 = df_input11.dropna(axis=0)

#print first 6 rows to check the data
print(df_input11[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set11 = df_input11.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set11 = df_input11.drop(training_set11.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set11 = df_input11.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features11 = training_set11.copy()
valid_features11 = valid_set11.copy()
test_features11 = test_set11.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels11 = training_features11.pop('NUM_COLLISIONS')
test_labels11 = test_features11.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features11.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels11 = training_labels11/SCALE_NUM_COLLISIONS
test_labels11 = test_labels11/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features11))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_3 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_3.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  fog  \
2807    7  62.2  1007.9    8.8  12.3   18.1  22.9  66.0  57.9  0.01    1   
2288    3  59.8  1014.3    9.6  12.3   15.9  21.0  66.0  51.1  0.00    0   
3995    2  48.3  1010.6    3.4  17.3   25.1  35.0  54.0  41.0  0.57    1   
2333    1  37.8  1018.6    9.9  13.3   17.1  21.0  46.0  25.0  0.00    0   
4127    4  37.7  1017.9    8.2  19.9   27.0  42.9  51.1  30.0  0.24    0   
2973    1  68.3  1020.1   10.0  11.2   19.0  22.0  75.0  50.0  0.00    0   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [49]:
%%time
history = dnn_model_3.fit(
    training_features11,
    training_labels11,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 15.9 s, sys: 745 ms, total: 16.6 s
Wall time: 16.3 s


In [50]:
# evaluate the mean absolute error for the model
dnn_model_3_results = dnn_model_3.evaluate(test_features11, test_labels11, verbose=0)
print(dnn_model_3_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_3 = dnn_model_3.predict(valid_features11)*1200 #scale factor

#create a copy of the validation set
valid_set11a = valid_set11.copy()

#add a column for the predicted number of collisions using our model
valid_set11a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_3.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set11a)



0.1623493731021881
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
      day  temp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  fog  \
3402    5  42.4  1026.5   10.0  13.4   21.0  28.9  50.0  28.0  0.00    0   
1643    3  58.8  1026.2   10.0   8.9   14.0  22.0  64.9  51.1  0.00    0   
4205    2  43.3  1014.5    6.6  16.0   28.0  39.0  48.0  28.9  0.35    0   
3736    5  38.4  1015.4    9.9  10.7   22.9  32.1  52.0  27.0  0.01    0   
3321    3  38.1  1018.9   10.0  14.7   20.0  27.0  46.0  21.9  0.00    0   
...   ...   ...     ...    ...   ...    ...   ...   ...   ...   ...  ...   
4357    4  38.9  1010.4    4.0  22.0   34.0  45.1  46.9  26.1  0.20    0   
3483    7  51.3  1017.2   10.0  14.2   21.0  28.0  57.0  37.9  0.00    0   
883     7  63.8  1015.4    9.0   6.4   12.0  15.9  72.0  55.4  0.22    0   
1273    6  75.7  1023.6    7.3   3.2   13.0  18.1  84.0  62.6  0.01    1   
2071    5  70.8  1008.1    7.9  10.9   15.9  21.0  80.1  64.0  0.18    0   

      NUM_COLLISIONS NUM_COLLI

###Remove Mean Sea Level Pressure

In [51]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data12 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers12 = ["day", "temp", "dewp", "visib", "wdsp", "mxpsd", "gust", "max", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input12 = pd.concat(df_input_data12, axis=1, keys=df_input_headers12)

#drop the rows with NaN values (regression can't handle this)
df_input12 = df_input12.dropna(axis=0)

#print first 6 rows to check the data
print(df_input12[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set12 = df_input12.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set12 = df_input12.drop(training_set12.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set12 = df_input12.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features12 = training_set12.copy()
valid_features12 = valid_set12.copy()
test_features12 = test_set12.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels12 = training_features12.pop('NUM_COLLISIONS')
test_labels12 = test_features12.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features12.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels12 = training_labels12/SCALE_NUM_COLLISIONS
test_labels12 = test_labels12/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features12))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_4 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_4.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp  visib  wdsp  mxpsd  gust   max   min  prcp  fog  \
2807    7  62.2  56.2    8.8  12.3   18.1  22.9  66.0  57.9  0.01    1   
2288    3  59.8  55.0    9.6  12.3   15.9  21.0  66.0  51.1  0.00    0   
3995    2  48.3  45.7    3.4  17.3   25.1  35.0  54.0  41.0  0.57    1   
2333    1  37.8  31.6    9.9  13.3   17.1  21.0  46.0  25.0  0.00    0   
4127    4  37.7  24.6    8.2  19.9   27.0  42.9  51.1  30.0  0.24    0   
2973    1  68.3  64.1   10.0  11.2   19.0  22.0  75.0  50.0  0.00    0   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [52]:
%%time
history = dnn_model_4.fit(
    training_features12,
    training_labels12,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 15.8 s, sys: 766 ms, total: 16.6 s
Wall time: 16.6 s


In [53]:
# evaluate the mean absolute error for the model
dnn_model_4_results = dnn_model_4.evaluate(test_features12, test_labels12, verbose=0)
print(dnn_model_4_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_4 = dnn_model_4.predict(valid_features12)*1200 #scale factor

#create a copy of the validation set
valid_set12a = valid_set12.copy()

#add a column for the predicted number of collisions using our model
valid_set12a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_4.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set12a)



0.1399620622396469
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
      day  temp  dewp  visib  wdsp  mxpsd  gust   max   min  prcp  fog  \
1504    6  41.2  37.6    9.1   7.5   14.0  20.0  48.9  37.9  0.10    0   
2408    7  69.4  59.9    9.5  11.8   17.1  22.9  75.0  64.0  0.00    0   
3567    7  39.4  34.8    8.3  12.4   21.0  29.9  45.0  33.1  0.55    0   
1720    4  61.0  61.0    4.2   8.6   15.0  22.0  66.9  54.0  0.03    1   
2977    6  61.4  58.0    6.5  11.1   19.0  28.0  66.9  52.0  0.08    0   
...   ...   ...   ...    ...   ...    ...   ...   ...   ...   ...  ...   
4357    4  38.9  36.3    4.0  22.0   34.0  45.1  46.9  26.1  0.20    0   
3720    7  56.5  48.2   10.0  15.5   22.0  25.1  63.0  51.1  0.00    0   
883     7  63.8  50.3    9.0   6.4   12.0  15.9  72.0  55.4  0.22    0   
1273    6  75.7  65.2    7.3   3.2   13.0  18.1  84.0  62.6  0.01    1   
3173    4  30.9  10.8   10.0  14.5   20.0  27.0  37.0  25.0  0.00    0   

      NUM_COLLISIONS NUM_COLLISIONS_predicted  
1504  

###Remove Maximum Temperature

In [54]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data13 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"],
                   df_shuffle["gust"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"], df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers13 = ["day", "temp", "dewp", "slp", "visib", "wdsp", "mxpsd", "gust", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input13 = pd.concat(df_input_data13, axis=1, keys=df_input_headers13)

#drop the rows with NaN values (regression can't handle this)
df_input13 = df_input13.dropna(axis=0)

#print first 6 rows to check the data
print(df_input13[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set13 = df_input13.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set13 = df_input13.drop(training_set13.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set13 = df_input13.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features13 = training_set13.copy()
valid_features13 = valid_set13.copy()
test_features13 = test_set13.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels13 = training_features13.pop('NUM_COLLISIONS')
test_labels13 = test_features13.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features13.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels13 = training_labels13/SCALE_NUM_COLLISIONS
test_labels13 = test_labels13/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features13))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_5 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_5.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   min  prcp  fog  \
2807    7  62.2  56.2  1007.9    8.8  12.3   18.1  22.9  57.9  0.01    1   
2288    3  59.8  55.0  1014.3    9.6  12.3   15.9  21.0  51.1  0.00    0   
3995    2  48.3  45.7  1010.6    3.4  17.3   25.1  35.0  41.0  0.57    1   
2333    1  37.8  31.6  1018.6    9.9  13.3   17.1  21.0  25.0  0.00    0   
4127    4  37.7  24.6  1017.9    8.2  19.9   27.0  42.9  30.0  0.24    0   
2973    1  68.3  64.1  1020.1   10.0  11.2   19.0  22.0  50.0  0.00    0   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [55]:
%%time
history = dnn_model_5.fit(
    training_features13,
    training_labels13,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 17.2 s, sys: 788 ms, total: 17.9 s
Wall time: 18.1 s


In [56]:
# evaluate the mean absolute error for the model
dnn_model_5_results = dnn_model_5.evaluate(test_features13, test_labels13, verbose=0)
print(dnn_model_5_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_5 = dnn_model_5.predict(valid_features13)*1200 #scale factor

#create a copy of the validation set
valid_set13a = valid_set13.copy()

#add a column for the predicted number of collisions using our model
valid_set13a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_5.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set13a)



0.14132946729660034
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   min  prcp  fog  \
1504    6  41.2  37.6  1017.1    9.1   7.5   14.0  20.0  37.9  0.10    0   
2408    7  69.4  59.9  1014.4    9.5  11.8   17.1  22.9  64.0  0.00    0   
3567    7  39.4  34.8  1021.0    8.3  12.4   21.0  29.9  33.1  0.55    0   
1720    4  61.0  61.0  1005.3    4.2   8.6   15.0  22.0  54.0  0.03    1   
2977    6  61.4  58.0  1019.9    6.5  11.1   19.0  28.0  52.0  0.08    0   
...   ...   ...   ...     ...    ...   ...    ...   ...   ...   ...  ...   
4357    4  38.9  36.3  1010.4    4.0  22.0   34.0  45.1  26.1  0.20    0   
3720    7  56.5  48.2  1026.6   10.0  15.5   22.0  25.1  51.1  0.00    0   
883     7  63.8  50.3  1015.4    9.0   6.4   12.0  15.9  55.4  0.22    0   
1273    6  75.7  65.2  1023.6    7.3   3.2   13.0  18.1  62.6  0.01    1   
3173    4  30.9  10.8  1015.8   10.0  14.5   20.0  27.0  25.0  0.00    0   

      NUM_COLLISIONS NUM_COLL

###Remove Minimum Temperature

In [57]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data14 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"],
                   df_shuffle["gust"], df_shuffle["max"], df_shuffle["prcp"], df_shuffle["fog"], df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers14 = ["day", "temp", "dewp", "slp", "visib", "wdsp", "mxpsd", "gust", "max", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input14 = pd.concat(df_input_data14, axis=1, keys=df_input_headers14)

#drop the rows with NaN values (regression can't handle this)
df_input14 = df_input14.dropna(axis=0)

#print first 6 rows to check the data
print(df_input14[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set14 = df_input14.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set14 = df_input14.drop(training_set14.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set14 = df_input14.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features14 = training_set14.copy()
valid_features14 = valid_set14.copy()
test_features14 = test_set14.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels14 = training_features14.pop('NUM_COLLISIONS')
test_labels14 = test_features14.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features14.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels14 = training_labels14/SCALE_NUM_COLLISIONS
test_labels14 = test_labels14/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features14))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_6 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_6.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   max  prcp  fog  \
2807    7  62.2  56.2  1007.9    8.8  12.3   18.1  22.9  66.0  0.01    1   
2288    3  59.8  55.0  1014.3    9.6  12.3   15.9  21.0  66.0  0.00    0   
3995    2  48.3  45.7  1010.6    3.4  17.3   25.1  35.0  54.0  0.57    1   
2333    1  37.8  31.6  1018.6    9.9  13.3   17.1  21.0  46.0  0.00    0   
4127    4  37.7  24.6  1017.9    8.2  19.9   27.0  42.9  51.1  0.24    0   
2973    1  68.3  64.1  1020.1   10.0  11.2   19.0  22.0  75.0  0.00    0   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [58]:
%%time
history = dnn_model_6.fit(
    training_features14,
    training_labels14,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 16.1 s, sys: 789 ms, total: 16.9 s
Wall time: 16 s


In [59]:
# evaluate the mean absolute error for the model
dnn_model_6_results = dnn_model_6.evaluate(test_features14, test_labels14, verbose=0)
print(dnn_model_6_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_6 = dnn_model_6.predict(valid_features14)*1200 #scale factor

#create a copy of the validation set
valid_set14a = valid_set14.copy()

#add a column for the predicted number of collisions using our model
valid_set14a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_6.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set14a)



0.13199076056480408
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   max  prcp  fog  \
1504    6  41.2  37.6  1017.1    9.1   7.5   14.0  20.0  48.9  0.10    0   
2408    7  69.4  59.9  1014.4    9.5  11.8   17.1  22.9  75.0  0.00    0   
3567    7  39.4  34.8  1021.0    8.3  12.4   21.0  29.9  45.0  0.55    0   
1720    4  61.0  61.0  1005.3    4.2   8.6   15.0  22.0  66.9  0.03    1   
2977    6  61.4  58.0  1019.9    6.5  11.1   19.0  28.0  66.9  0.08    0   
...   ...   ...   ...     ...    ...   ...    ...   ...   ...   ...  ...   
4357    4  38.9  36.3  1010.4    4.0  22.0   34.0  45.1  46.9  0.20    0   
3720    7  56.5  48.2  1026.6   10.0  15.5   22.0  25.1  63.0  0.00    0   
883     7  63.8  50.3  1015.4    9.0   6.4   12.0  15.9  72.0  0.22    0   
1273    6  75.7  65.2  1023.6    7.3   3.2   13.0  18.1  84.0  0.01    1   
3173    4  30.9  10.8  1015.8   10.0  14.5   20.0  27.0  37.0  0.00    0   

      NUM_COLLISIONS NUM_COLL

###Remove Reported Fog

In [60]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data15 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"],
                   df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers15 = ["day", "temp", "dewp", "slp", "visib", "wdsp", "mxpsd", "gust", "max", "min", "prcp",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input15 = pd.concat(df_input_data15, axis=1, keys=df_input_headers15)

#drop the rows with NaN values (regression can't handle this)
df_input15 = df_input15.dropna(axis=0)

#print first 6 rows to check the data
print(df_input15[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set15 = df_input15.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set15 = df_input15.drop(training_set15.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set15 = df_input15.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features15 = training_set15.copy()
valid_features15 = valid_set15.copy()
test_features15 = test_set15.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels15 = training_features15.pop('NUM_COLLISIONS')
test_labels15 = test_features15.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features15.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels15 = training_labels15/SCALE_NUM_COLLISIONS
test_labels15 = test_labels15/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features15))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_7 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_7.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  \
2807    7  62.2  56.2  1007.9    8.8  12.3   18.1  22.9  66.0  57.9  0.01   
2288    3  59.8  55.0  1014.3    9.6  12.3   15.9  21.0  66.0  51.1  0.00   
3995    2  48.3  45.7  1010.6    3.4  17.3   25.1  35.0  54.0  41.0  0.57   
2333    1  37.8  31.6  1018.6    9.9  13.3   17.1  21.0  46.0  25.0  0.00   
4127    4  37.7  24.6  1017.9    8.2  19.9   27.0  42.9  51.1  30.0  0.24   
2973    1  68.3  64.1  1020.1   10.0  11.2   19.0  22.0  75.0  50.0  0.00   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [61]:
%%time
history = dnn_model_7.fit(
    training_features15,
    training_labels15,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 15.8 s, sys: 799 ms, total: 16.6 s
Wall time: 16 s


In [62]:
# evaluate the mean absolute error for the model
dnn_model_7_results = dnn_model_7.evaluate(test_features15, test_labels15, verbose=0)
print(dnn_model_7_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_7 = dnn_model_7.predict(valid_features15)*1200 #scale factor

#create a copy of the validation set
valid_set15a = valid_set15.copy()

#add a column for the predicted number of collisions using our model
valid_set15a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_7.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set15a)



0.14263370633125305
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  \
1504    6  41.2  37.6  1017.1    9.1   7.5   14.0  20.0  48.9  37.9  0.10   
2408    7  69.4  59.9  1014.4    9.5  11.8   17.1  22.9  75.0  64.0  0.00   
3567    7  39.4  34.8  1021.0    8.3  12.4   21.0  29.9  45.0  33.1  0.55   
1720    4  61.0  61.0  1005.3    4.2   8.6   15.0  22.0  66.9  54.0  0.03   
2977    6  61.4  58.0  1019.9    6.5  11.1   19.0  28.0  66.9  52.0  0.08   
...   ...   ...   ...     ...    ...   ...    ...   ...   ...   ...   ...   
4357    4  38.9  36.3  1010.4    4.0  22.0   34.0  45.1  46.9  26.1  0.20   
3720    7  56.5  48.2  1026.6   10.0  15.5   22.0  25.1  63.0  51.1  0.00   
883     7  63.8  50.3  1015.4    9.0   6.4   12.0  15.9  72.0  55.4  0.22   
1273    6  75.7  65.2  1023.6    7.3   3.2   13.0  18.1  84.0  62.6  0.01   
3173    4  30.9  10.8  1015.8   10.0  14.5   20.0  27.0  37.0  25.0  0.00   

      NUM_COLLISI

###Remove Mean Wind Speed

In [63]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data16 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["mxpsd"], df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers16 = ["day", "temp", "dewp", "slp", "visib", "mxpsd", "gust", "max", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input16 = pd.concat(df_input_data16, axis=1, keys=df_input_headers16)

#drop the rows with NaN values (regression can't handle this)
df_input16 = df_input16.dropna(axis=0)

#print first 6 rows to check the data
print(df_input16[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set16 = df_input16.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set16 = df_input16.drop(training_set16.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set16 = df_input16.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features16 = training_set16.copy()
valid_features16 = valid_set16.copy()
test_features16 = test_set16.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels16 = training_features16.pop('NUM_COLLISIONS')
test_labels16 = test_features16.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features16.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels16 = training_labels16/SCALE_NUM_COLLISIONS
test_labels16 = test_labels16/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features16))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_8 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_8.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp     slp  visib  mxpsd  gust   max   min  prcp  fog  \
2807    7  62.2  56.2  1007.9    8.8   18.1  22.9  66.0  57.9  0.01    1   
2288    3  59.8  55.0  1014.3    9.6   15.9  21.0  66.0  51.1  0.00    0   
3995    2  48.3  45.7  1010.6    3.4   25.1  35.0  54.0  41.0  0.57    1   
2333    1  37.8  31.6  1018.6    9.9   17.1  21.0  46.0  25.0  0.00    0   
4127    4  37.7  24.6  1017.9    8.2   27.0  42.9  51.1  30.0  0.24    0   
2973    1  68.3  64.1  1020.1   10.0   19.0  22.0  75.0  50.0  0.00    0   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [64]:
%%time
history = dnn_model_8.fit(
    training_features16,
    training_labels16,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 16.2 s, sys: 833 ms, total: 17 s
Wall time: 16.8 s


In [65]:
# evaluate the mean absolute error for the model
dnn_model_8_results = dnn_model_8.evaluate(test_features16, test_labels16, verbose=0)
print(dnn_model_8_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_8 = dnn_model_8.predict(valid_features16)*1200 #scale factor

#create a copy of the validation set
valid_set16a = valid_set16.copy()

#add a column for the predicted number of collisions using our model
valid_set16a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_8.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set16a)



0.13620688021183014
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
      day  temp  dewp     slp  visib  mxpsd  gust   max   min  prcp  fog  \
1504    6  41.2  37.6  1017.1    9.1   14.0  20.0  48.9  37.9  0.10    0   
2408    7  69.4  59.9  1014.4    9.5   17.1  22.9  75.0  64.0  0.00    0   
3567    7  39.4  34.8  1021.0    8.3   21.0  29.9  45.0  33.1  0.55    0   
1720    4  61.0  61.0  1005.3    4.2   15.0  22.0  66.9  54.0  0.03    1   
2977    6  61.4  58.0  1019.9    6.5   19.0  28.0  66.9  52.0  0.08    0   
...   ...   ...   ...     ...    ...    ...   ...   ...   ...   ...  ...   
4357    4  38.9  36.3  1010.4    4.0   34.0  45.1  46.9  26.1  0.20    0   
3720    7  56.5  48.2  1026.6   10.0   22.0  25.1  63.0  51.1  0.00    0   
883     7  63.8  50.3  1015.4    9.0   12.0  15.9  72.0  55.4  0.22    0   
1273    6  75.7  65.2  1023.6    7.3   13.0  18.1  84.0  62.6  0.01    1   
3173    4  30.9  10.8  1015.8   10.0   20.0  27.0  37.0  25.0  0.00    0   

      NUM_COLLISIONS NUM_COLL

###Remove Maximum Sustained Wind Speed

In [66]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data17 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers17 = ["day", "temp", "dewp", "slp", "visib", "wdsp", "gust", "max", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input17 = pd.concat(df_input_data17, axis=1, keys=df_input_headers17)

#drop the rows with NaN values (regression can't handle this)
df_input17 = df_input17.dropna(axis=0)

#print first 6 rows to check the data
print(df_input17[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set17 = df_input17.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set17 = df_input17.drop(training_set17.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set17 = df_input17.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features17 = training_set17.copy()
valid_features17 = valid_set17.copy()
test_features17 = test_set17.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels17 = training_features17.pop('NUM_COLLISIONS')
test_labels17 = test_features17.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features17.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels17 = training_labels17/SCALE_NUM_COLLISIONS
test_labels17 = test_labels17/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features17))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_9 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_9.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp     slp  visib  wdsp  gust   max   min  prcp  fog  \
2807    7  62.2  56.2  1007.9    8.8  12.3  22.9  66.0  57.9  0.01    1   
2288    3  59.8  55.0  1014.3    9.6  12.3  21.0  66.0  51.1  0.00    0   
3995    2  48.3  45.7  1010.6    3.4  17.3  35.0  54.0  41.0  0.57    1   
2333    1  37.8  31.6  1018.6    9.9  13.3  21.0  46.0  25.0  0.00    0   
4127    4  37.7  24.6  1017.9    8.2  19.9  42.9  51.1  30.0  0.24    0   
2973    1  68.3  64.1  1020.1   10.0  11.2  22.0  75.0  50.0  0.00    0   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [67]:
%%time
history = dnn_model_9.fit(
    training_features17,
    training_labels17,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 15.8 s, sys: 796 ms, total: 16.6 s
Wall time: 15.9 s


In [68]:
# evaluate the mean absolute error for the model
dnn_model_9_results = dnn_model_9.evaluate(test_features17, test_labels17, verbose=0)
print(dnn_model_9_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_9 = dnn_model_9.predict(valid_features17)*1200 #scale factor

#create a copy of the validation set
valid_set17a = valid_set17.copy()

#add a column for the predicted number of collisions using our model
valid_set17a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_9.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set17a)



0.14368751645088196
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
      day  temp  dewp     slp  visib  wdsp  gust   max   min  prcp  fog  \
1504    6  41.2  37.6  1017.1    9.1   7.5  20.0  48.9  37.9  0.10    0   
2408    7  69.4  59.9  1014.4    9.5  11.8  22.9  75.0  64.0  0.00    0   
3567    7  39.4  34.8  1021.0    8.3  12.4  29.9  45.0  33.1  0.55    0   
1720    4  61.0  61.0  1005.3    4.2   8.6  22.0  66.9  54.0  0.03    1   
2977    6  61.4  58.0  1019.9    6.5  11.1  28.0  66.9  52.0  0.08    0   
...   ...   ...   ...     ...    ...   ...   ...   ...   ...   ...  ...   
4357    4  38.9  36.3  1010.4    4.0  22.0  45.1  46.9  26.1  0.20    0   
3720    7  56.5  48.2  1026.6   10.0  15.5  25.1  63.0  51.1  0.00    0   
883     7  63.8  50.3  1015.4    9.0   6.4  15.9  72.0  55.4  0.22    0   
1273    6  75.7  65.2  1023.6    7.3   3.2  18.1  84.0  62.6  0.01    1   
3173    4  30.9  10.8  1015.8   10.0  14.5  27.0  37.0  25.0  0.00    0   

      NUM_COLLISIONS NUM_COLLISIONS_predi

###Remove Maximum Wind Gust

In [69]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data18 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers18 = ["day", "temp", "dewp", "slp", "visib", "wdsp", "mxpsd", "max", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input18 = pd.concat(df_input_data18, axis=1, keys=df_input_headers18)

#drop the rows with NaN values (regression can't handle this)
df_input18 = df_input18.dropna(axis=0)

#print first 6 rows to check the data
print(df_input18[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set18 = df_input18.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set18 = df_input18.drop(training_set18.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set18 = df_input18.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features18 = training_set18.copy()
valid_features18 = valid_set18.copy()
test_features18 = test_set18.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels18 = training_features18.pop('NUM_COLLISIONS')
test_labels18 = test_features18.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features18.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels18 = training_labels18/SCALE_NUM_COLLISIONS
test_labels18 = test_labels18/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features18))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_10 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_10.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp     slp  visib  wdsp  mxpsd   max   min  prcp  fog  \
1785    3  62.9  51.9  1019.8   10.0   8.2   15.0  73.9  57.0  0.57    0   
2807    7  62.2  56.2  1007.9    8.8  12.3   18.1  66.0  57.9  0.01    1   
2476    5  66.9  63.6  1017.1    7.8   8.7   17.1  77.0  61.0  0.00    1   
2288    3  59.8  55.0  1014.3    9.6  12.3   15.9  66.0  51.1  0.00    0   
1697    2  55.3  48.6  1022.7   10.0   9.6   15.0  64.0  46.0  0.00    0   
640     1  32.7  23.9  1021.3    9.7   4.6   11.1  43.0  24.1  0.00    0   

      NUM_COLLISIONS  
1785             686  
2807             206  
2476             752  
2288             655  
1697             720  
640              706  


In [70]:
%%time
history = dnn_model_10.fit(
    training_features18,
    training_labels18,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 22.3 s, sys: 1.12 s, total: 23.4 s
Wall time: 27.9 s


In [71]:
# evaluate the mean absolute error for the model
dnn_model_10_results = dnn_model_10.evaluate(test_features18, test_labels18, verbose=0)
print(dnn_model_10_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_10 = dnn_model_10.predict(valid_features18)*1200 #scale factor

#create a copy of the validation set
valid_set18a = valid_set18.copy()

#add a column for the predicted number of collisions using our model
valid_set18a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_10.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set18a)



0.1391804963350296
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
      day  temp  dewp     slp  visib  wdsp  mxpsd   max   min  prcp  fog  \
2613    3  64.8  58.1  1012.1    9.3   8.7   17.1  73.9  53.1  0.01    1   
2236    1  32.1  15.5  1027.6   10.0   9.0   15.9  37.0  26.1  0.00    0   
1220    6  42.6  27.1  1024.5    9.9   2.8   13.0  46.9  33.8  0.00    0   
3195    1  52.6  45.8  1019.0    8.2  15.3   20.0  61.0  34.0  0.01    0   
3218    6  50.7  49.8  1013.7    3.6  15.2   20.0  59.0  50.0  0.07    1   
...   ...   ...   ...     ...    ...   ...    ...   ...   ...   ...  ...   
2165    2  56.1  53.0  1008.2    5.9   8.0   15.9  66.0  46.9  0.01    1   
3846    1  54.9  45.0  1017.4   10.0  16.0   22.9  59.0  51.1  0.00    0   
1096    4  54.0  39.9  1025.2   10.0   6.9   13.0  66.9  45.0  0.00    0   
2688    7  63.1  56.2  1020.3    9.4   9.1   18.1  78.1  50.0  0.00    0   
431     4  70.4  54.8  1027.4   10.0   3.2    9.9  80.1  62.1  0.00    0   

      NUM_COLLISIONS NUM_COLLI

###Remove Total Precipitation

In [72]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data19 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"],
                   df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["fog"], df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers19 = ["day", "temp", "dewp", "slp", "visib", "wdsp", "mxpsd", "gust", "max", "min", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input19 = pd.concat(df_input_data19, axis=1, keys=df_input_headers19)

#drop the rows with NaN values (regression can't handle this)
df_input19 = df_input19.dropna(axis=0)

#print first 6 rows to check the data
print(df_input19[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set19 = df_input19.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set19 = df_input19.drop(training_set19.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set19 = df_input19.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features19 = training_set19.copy()
valid_features19 = valid_set19.copy()
test_features19 = test_set19.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels19 = training_features19.pop('NUM_COLLISIONS')
test_labels19 = test_features19.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features19.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels19 = training_labels19/SCALE_NUM_COLLISIONS
test_labels19 = test_labels19/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features19))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_11 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_11.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   max   min  fog  \
2807    7  62.2  56.2  1007.9    8.8  12.3   18.1  22.9  66.0  57.9    1   
2288    3  59.8  55.0  1014.3    9.6  12.3   15.9  21.0  66.0  51.1    0   
3995    2  48.3  45.7  1010.6    3.4  17.3   25.1  35.0  54.0  41.0    1   
2333    1  37.8  31.6  1018.6    9.9  13.3   17.1  21.0  46.0  25.0    0   
4127    4  37.7  24.6  1017.9    8.2  19.9   27.0  42.9  51.1  30.0    0   
2973    1  68.3  64.1  1020.1   10.0  11.2   19.0  22.0  75.0  50.0    0   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [73]:
%%time
history = dnn_model_11.fit(
    training_features19,
    training_labels19,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 16.4 s, sys: 769 ms, total: 17.2 s
Wall time: 17.1 s


In [74]:
# evaluate the mean absolute error for the model
dnn_model_11_results = dnn_model_11.evaluate(test_features19, test_labels19, verbose=0)
print(dnn_model_11_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_11 = dnn_model_11.predict(valid_features19)*1200 #scale factor

#create a copy of the validation set
valid_set19a = valid_set19.copy()

#add a column for the predicted number of collisions using our model
valid_set19a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_11.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set19a)



0.13887201249599457
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
      day  temp  dewp     slp  visib  wdsp  mxpsd  gust   max   min  fog  \
3271    1  62.3  58.2  1011.6    8.8  13.8   20.0  27.0  68.0  55.0    0   
3709    7  48.8  39.8  1014.2    9.6  11.9   22.0  28.9  54.0  44.6    0   
3135    7  49.6  47.9  1020.7    6.0  15.3   19.0  28.0  54.0  44.1    1   
4263    4  43.7  39.1  1021.6    5.3  16.4   28.9  36.9  50.0  35.1    1   
1746    4  42.7  35.1  1012.7    9.8   6.4   15.0  22.9  46.9  37.0    0   
...   ...   ...   ...     ...    ...   ...    ...   ...   ...   ...  ...   
2771    2  63.3  59.7  1021.0    8.7  14.1   18.1  22.9  66.9  61.0    0   
2355    2  37.1  22.6  1013.5   10.0  12.2   17.1  24.1  48.0  32.0    0   
2506    5  53.6  44.3  1015.1   10.0  12.1   17.1  22.9  63.0  48.9    0   
3595    7  37.8  34.1  1026.4    9.1  13.4   22.0  28.0  46.0  30.0    0   
2922    3  42.1  38.5  1012.8    5.0  12.3   19.0  25.1  48.0  28.0    1   

      NUM_COLLISIONS NUM_COLL

###Remove Mean Visibility

In [75]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data20 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers20 = ["day", "temp", "dewp", "slp", "wdsp", "mxpsd", "gust", "max", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input20 = pd.concat(df_input_data20, axis=1, keys=df_input_headers20)

#drop the rows with NaN values (regression can't handle this)
df_input20 = df_input20.dropna(axis=0)

#print first 6 rows to check the data
print(df_input20[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set20 = df_input20.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set20 = df_input20.drop(training_set20.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set20 = df_input20.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features20 = training_set20.copy()
valid_features20 = valid_set20.copy()
test_features20 = test_set20.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels20 = training_features20.pop('NUM_COLLISIONS')
test_labels20 = test_features20.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features20.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels20 = training_labels20/SCALE_NUM_COLLISIONS
test_labels20 = test_labels20/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features20))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_12 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_12.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp     slp  wdsp  mxpsd  gust   max   min  prcp  fog  \
2807    7  62.2  56.2  1007.9  12.3   18.1  22.9  66.0  57.9  0.01    1   
2288    3  59.8  55.0  1014.3  12.3   15.9  21.0  66.0  51.1  0.00    0   
3995    2  48.3  45.7  1010.6  17.3   25.1  35.0  54.0  41.0  0.57    1   
2333    1  37.8  31.6  1018.6  13.3   17.1  21.0  46.0  25.0  0.00    0   
4127    4  37.7  24.6  1017.9  19.9   27.0  42.9  51.1  30.0  0.24    0   
2973    1  68.3  64.1  1020.1  11.2   19.0  22.0  75.0  50.0  0.00    0   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [76]:
%%time
history = dnn_model_12.fit(
    training_features20,
    training_labels20,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 15.7 s, sys: 834 ms, total: 16.5 s
Wall time: 16.2 s


In [77]:
# evaluate the mean absolute error for the model
dnn_model_12_results = dnn_model_12.evaluate(test_features20, test_labels20, verbose=0)
print(dnn_model_12_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_12 = dnn_model_12.predict(valid_features20)*1200 #scale factor

#create a copy of the validation set
valid_set20a = valid_set20.copy()

#add a column for the predicted number of collisions using our model
valid_set20a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_12.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set20a)



0.14489327371120453
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
      day  temp  dewp     slp  wdsp  mxpsd  gust   max   min  prcp  fog  \
1504    6  41.2  37.6  1017.1   7.5   14.0  20.0  48.9  37.9  0.10    0   
2408    7  69.4  59.9  1014.4  11.8   17.1  22.9  75.0  64.0  0.00    0   
3567    7  39.4  34.8  1021.0  12.4   21.0  29.9  45.0  33.1  0.55    0   
1720    4  61.0  61.0  1005.3   8.6   15.0  22.0  66.9  54.0  0.03    1   
2977    6  61.4  58.0  1019.9  11.1   19.0  28.0  66.9  52.0  0.08    0   
...   ...   ...   ...     ...   ...    ...   ...   ...   ...   ...  ...   
4357    4  38.9  36.3  1010.4  22.0   34.0  45.1  46.9  26.1  0.20    0   
3720    7  56.5  48.2  1026.6  15.5   22.0  25.1  63.0  51.1  0.00    0   
883     7  63.8  50.3  1015.4   6.4   12.0  15.9  72.0  55.4  0.22    0   
1273    6  75.7  65.2  1023.6   3.2   13.0  18.1  84.0  62.6  0.01    1   
3173    4  30.9  10.8  1015.8  14.5   20.0  27.0  37.0  25.0  0.00    0   

      NUM_COLLISIONS NUM_COLLISIONS_predi

###Remove Day of Week

In [78]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data21 = [df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["slp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"], df_shuffle["max"], df_shuffle["min"], df_shuffle["prcp"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers21 = ["temp", "dewp", "slp", "visib", "wdsp", "mxpsd", "gust", "max", "min", "prcp", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input21 = pd.concat(df_input_data21, axis=1, keys=df_input_headers21)

#drop the rows with NaN values (regression can't handle this)
df_input21 = df_input21.dropna(axis=0)

#print first 6 rows to check the data
print(df_input21[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set21 = df_input21.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set21 = df_input21.drop(training_set21.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set21 = df_input21.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features21 = training_set21.copy()
valid_features21 = valid_set21.copy()
test_features21 = test_set21.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels21 = training_features21.pop('NUM_COLLISIONS')
test_labels21 = test_features21.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features21.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels21 = training_labels21/SCALE_NUM_COLLISIONS
test_labels21 = test_labels21/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features21))

# Normalisation layer (11 inputs), 2 layers of 48, with 1 output.
dnn_model_13 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_13.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      temp  dewp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  fog  \
2807  62.2  56.2  1007.9    8.8  12.3   18.1  22.9  66.0  57.9  0.01    1   
2288  59.8  55.0  1014.3    9.6  12.3   15.9  21.0  66.0  51.1  0.00    0   
3995  48.3  45.7  1010.6    3.4  17.3   25.1  35.0  54.0  41.0  0.57    1   
2333  37.8  31.6  1018.6    9.9  13.3   17.1  21.0  46.0  25.0  0.00    0   
4127  37.7  24.6  1017.9    8.2  19.9   27.0  42.9  51.1  30.0  0.24    0   
2973  68.3  64.1  1020.1   10.0  11.2   19.0  22.0  75.0  50.0  0.00    0   

      NUM_COLLISIONS  
2807             206  
2288             655  
3995             608  
2333             238  
4127             252  
2973             521  


In [79]:
%%time
history = dnn_model_13.fit(
    training_features21,
    training_labels21,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 15.7 s, sys: 787 ms, total: 16.5 s
Wall time: 16.3 s


In [80]:
# evaluate the mean absolute error for the model
dnn_model_13_results = dnn_model_13.evaluate(test_features21, test_labels21, verbose=0)
print(dnn_model_13_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_13 = dnn_model_13.predict(valid_features21)*1200 #scale factor

#create a copy of the validation set
valid_set21a = valid_set21.copy()

#add a column for the predicted number of collisions using our model
valid_set21a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_13.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set21a)



0.143428772687912
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
      temp  dewp     slp  visib  wdsp  mxpsd  gust   max   min  prcp  fog  \
1504  41.2  37.6  1017.1    9.1   7.5   14.0  20.0  48.9  37.9  0.10    0   
2408  69.4  59.9  1014.4    9.5  11.8   17.1  22.9  75.0  64.0  0.00    0   
3567  39.4  34.8  1021.0    8.3  12.4   21.0  29.9  45.0  33.1  0.55    0   
1720  61.0  61.0  1005.3    4.2   8.6   15.0  22.0  66.9  54.0  0.03    1   
2977  61.4  58.0  1019.9    6.5  11.1   19.0  28.0  66.9  52.0  0.08    0   
...    ...   ...     ...    ...   ...    ...   ...   ...   ...   ...  ...   
4357  38.9  36.3  1010.4    4.0  22.0   34.0  45.1  46.9  26.1  0.20    0   
3720  56.5  48.2  1026.6   10.0  15.5   22.0  25.1  63.0  51.1  0.00    0   
883   63.8  50.3  1015.4    9.0   6.4   12.0  15.9  72.0  55.4  0.22    0   
1273  75.7  65.2  1023.6    7.3   3.2   13.0  18.1  84.0  62.6  0.01    1   
3173  30.9  10.8  1015.8   10.0  14.5   20.0  27.0  37.0  25.0  0.00    0   

      NUM_COLLISION

###DNN Model with Significant Variables from Linear Model

In [81]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data22 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["wdsp"], df_shuffle["mxpsd"], df_shuffle["gust"], df_shuffle["prcp"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers22 = ["day", "temp", "dewp", "visib", "wdsp", "mxpsd", "gust", "prcp",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input22 = pd.concat(df_input_data22, axis=1, keys=df_input_headers22)

#drop the rows with NaN values (regression can't handle this)
df_input22 = df_input22.dropna(axis=0)

#print first 6 rows to check the data
print(df_input22[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set22 = df_input22.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set22 = df_input22.drop(training_set22.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set22 = df_input22.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features22 = training_set22.copy()
valid_features22 = valid_set22.copy()
test_features22 = test_set22.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels22 = training_features22.pop('NUM_COLLISIONS')
test_labels22 = test_features22.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features22.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels22 = training_labels22/SCALE_NUM_COLLISIONS
test_labels22 = test_labels22/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features22))

# Normalisation layer (8 inputs), 2 layers of 48, with 1 output.
dnn_model_14 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_14.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp  visib  wdsp  mxpsd  gust  prcp  NUM_COLLISIONS
2807    7  62.2  56.2    8.8  12.3   18.1  22.9  0.01             206
2288    3  59.8  55.0    9.6  12.3   15.9  21.0  0.00             655
3995    2  48.3  45.7    3.4  17.3   25.1  35.0  0.57             608
2333    1  37.8  31.6    9.9  13.3   17.1  21.0  0.00             238
4127    4  37.7  24.6    8.2  19.9   27.0  42.9  0.24             252
2973    1  68.3  64.1   10.0  11.2   19.0  22.0  0.00             521


In [82]:
%%time
history = dnn_model_14.fit(
    training_features22,
    training_labels22,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 16.7 s, sys: 813 ms, total: 17.5 s
Wall time: 16.9 s


In [83]:
# evaluate the mean absolute error for the model
dnn_model_14_results = dnn_model_14.evaluate(test_features22, test_labels22, verbose=0)
print(dnn_model_14_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_14 = dnn_model_14.predict(valid_features22)*1200 #scale factor

#create a copy of the validation set
valid_set22a = valid_set22.copy()

#add a column for the predicted number of collisions using our model
valid_set22a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_14.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set22a)


0.1338409036397934
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
      day  temp  dewp  visib  wdsp  mxpsd  gust  prcp  NUM_COLLISIONS  \
1504    6  41.2  37.6    9.1   7.5   14.0  20.0  0.10             181   
2408    7  69.4  59.9    9.5  11.8   17.1  22.9  0.00             519   
3567    7  39.4  34.8    8.3  12.4   21.0  29.9  0.55             472   
1720    4  61.0  61.0    4.2   8.6   15.0  22.0  0.03             279   
2977    6  61.4  58.0    6.5  11.1   19.0  28.0  0.08             328   
...   ...   ...   ...    ...   ...    ...   ...   ...             ...   
4357    4  38.9  36.3    4.0  22.0   34.0  45.1  0.20             233   
3720    7  56.5  48.2   10.0  15.5   22.0  25.1  0.00             432   
883     7  63.8  50.3    9.0   6.4   12.0  15.9  0.22             480   
1273    6  75.7  65.2    7.3   3.2   13.0  18.1  0.01             474   
3173    4  30.9  10.8   10.0  14.5   20.0  27.0  0.00             666   

     NUM_COLLISIONS_predicted  
1504      [439.2330322265625]  
24

###Remove All Variables Whose Removal Decreased Mean Absolute Error

In [84]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data23 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["mxpsd"], df_shuffle["max"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers23 = ["day", "temp", "dewp", "visib", "mxpsd", "max", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input23 = pd.concat(df_input_data23, axis=1, keys=df_input_headers23)

#drop the rows with NaN values (regression can't handle this)
df_input23 = df_input23.dropna(axis=0)

#print first 6 rows to check the data
print(df_input23[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set23 = df_input23.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set23 = df_input23.drop(training_set23.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set23 = df_input23.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features23 = training_set23.copy()
valid_features23 = valid_set23.copy()
test_features23 = test_set23.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels23 = training_features23.pop('NUM_COLLISIONS')
test_labels23 = test_features23.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features23.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels23 = training_labels23/SCALE_NUM_COLLISIONS
test_labels23 = test_labels23/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features23))

# Normalisation layer (7 inputs), 2 layers of 48, with 1 output.
dnn_model_15 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_15.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp  visib  mxpsd   max  fog  NUM_COLLISIONS
1785    3  62.9  51.9   10.0   15.0  73.9    0             686
2807    7  62.2  56.2    8.8   18.1  66.0    1             206
2476    5  66.9  63.6    7.8   17.1  77.0    1             752
2288    3  59.8  55.0    9.6   15.9  66.0    0             655
1697    2  55.3  48.6   10.0   15.0  64.0    0             720
640     1  32.7  23.9    9.7   11.1  43.0    0             706


In [85]:
%%time
history = dnn_model_15.fit(
    training_features23,
    training_labels23,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 22.3 s, sys: 1.09 s, total: 23.4 s
Wall time: 27 s


In [86]:
# evaluate the mean absolute error for the model
dnn_model_15_results = dnn_model_15.evaluate(test_features23, test_labels23, verbose=0)
print(dnn_model_15_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_15 = dnn_model_15.predict(valid_features23)*1200 #scale factor

#create a copy of the validation set
valid_set23a = valid_set23.copy()

#add a column for the predicted number of collisions using our model
valid_set23a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_15.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set23a)


0.13062207400798798
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
      day  temp  dewp  visib  mxpsd   max  fog  NUM_COLLISIONS  \
3727    4  24.9   4.8   10.0   22.9  44.1    0             412   
3834    7  51.9  51.2    2.1   22.9  55.9    1             505   
2859    6  44.9  36.6    9.4   18.1  55.0    0             549   
3822    2  32.6  25.6    8.4   22.9  52.0    0             613   
3578    2  60.5  55.1    8.7   21.0  64.9    0             715   
...   ...   ...   ...    ...    ...   ...  ...             ...   
3537    4  46.6  41.2   10.0   21.0  55.9    0             197   
2914    4  29.9  11.8   10.0   19.0  36.0    0             625   
2765    5  61.7  58.6    4.0   18.1  66.0    1             708   
2248    7  71.5  70.6    7.0   15.9  77.0    1             284   
2010    3  41.1  38.4    6.5   15.9  48.9    0             583   

     NUM_COLLISIONS_predicted  
3727      [620.3088989257812]  
3834     [371.98004150390625]  
2859      [535.9517822265625]  
3822      [487.804779

###Remove All Variables Whose Removal Decreased Mean Absolute Error AND Increase Neurons in Dense Layer

In [87]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data24 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["mxpsd"], df_shuffle["max"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers24 = ["day", "temp", "dewp", "visib", "mxpsd", "max", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input24 = pd.concat(df_input_data24, axis=1, keys=df_input_headers24)

#drop the rows with NaN values (regression can't handle this)
df_input24 = df_input24.dropna(axis=0)

#print first 6 rows to check the data
print(df_input24[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set24 = df_input24.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set24 = df_input24.drop(training_set24.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set24 = df_input24.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features24 = training_set24.copy()
valid_features24 = valid_set24.copy()
test_features24 = test_set24.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels24 = training_features24.pop('NUM_COLLISIONS')
test_labels24 = test_features24.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features24.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels24 = training_labels24/SCALE_NUM_COLLISIONS
test_labels24 = test_labels24/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features24))

# Normalisation layer (7 inputs), increase to 2 layers of 100, with 1 output.
dnn_model_16 = keras.Sequential([
      normaliser,
      layers.Dense(100, activation='relu'),
      layers.Dense(100, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_16.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp  visib  mxpsd   max  fog  NUM_COLLISIONS
1785    3  62.9  51.9   10.0   15.0  73.9    0             686
2807    7  62.2  56.2    8.8   18.1  66.0    1             206
2476    5  66.9  63.6    7.8   17.1  77.0    1             752
2288    3  59.8  55.0    9.6   15.9  66.0    0             655
1697    2  55.3  48.6   10.0   15.0  64.0    0             720
640     1  32.7  23.9    9.7   11.1  43.0    0             706


In [88]:
%%time
history = dnn_model_16.fit(
    training_features24,
    training_labels24,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 23.3 s, sys: 1.16 s, total: 24.5 s
Wall time: 29 s


In [89]:
# evaluate the mean absolute error for the model
dnn_model_16_results = dnn_model_16.evaluate(test_features24, test_labels24, verbose=0)
print(dnn_model_16_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_16 = dnn_model_16.predict(valid_features24)*1200 #scale factor

#create a copy of the validation set
valid_set24a = valid_set24.copy()

#add a column for the predicted number of collisions using our model
valid_set24a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_16.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set24a)


0.13369134068489075
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
      day  temp  dewp  visib  mxpsd   max  fog  NUM_COLLISIONS  \
3727    4  24.9   4.8   10.0   22.9  44.1    0             412   
3834    7  51.9  51.2    2.1   22.9  55.9    1             505   
2859    6  44.9  36.6    9.4   18.1  55.0    0             549   
3822    2  32.6  25.6    8.4   22.9  52.0    0             613   
3578    2  60.5  55.1    8.7   21.0  64.9    0             715   
...   ...   ...   ...    ...    ...   ...  ...             ...   
3537    4  46.6  41.2   10.0   21.0  55.9    0             197   
2914    4  29.9  11.8   10.0   19.0  36.0    0             625   
2765    5  61.7  58.6    4.0   18.1  66.0    1             708   
2248    7  71.5  70.6    7.0   15.9  77.0    1             284   
2010    3  41.1  38.4    6.5   15.9  48.9    0             583   

     NUM_COLLISIONS_predicted  
3727      [526.2708740234375]  
3834      [448.9442443847656]  
2859     [445.73138427734375]  
3822      [426.280120

###Remove All Variables Whose Removal Decreased Mean Absolute Error AND Increase Number of Layers

In [90]:
# create an input dataframe from the shuffled dataframe

#get the data
df_input_data25 = [df_shuffle["day"], df_shuffle["temp"], df_shuffle["dewp"], df_shuffle["visib"], df_shuffle["mxpsd"], df_shuffle["max"], df_shuffle["fog"],
                 df_shuffle["NUM_COLLISIONS"]]

# create headers
df_input_headers25 = ["day", "temp", "dewp", "visib", "mxpsd", "max", "fog",
                    "NUM_COLLISIONS"]

# create a dataframe containing the data and headers
df_input25 = pd.concat(df_input_data25, axis=1, keys=df_input_headers25)

#drop the rows with NaN values (regression can't handle this)
df_input25 = df_input25.dropna(axis=0)

#print first 6 rows to check the data
print(df_input25[:6])

# create a training set, validation set, and a test set for the model. Sample 80% for the training set and remaining 20% for the test set

#create a training set with fraction 80% of data, set random_state=0 for reproducibility
training_set25 = df_input25.sample(frac=0.8, random_state=0)

#create a test set with the remaining 20% of data, by dropping the 80% in the training set based on the index
test_set25 = df_input25.drop(training_set25.index)

#Also sample 5% for the validation set, set random_state=0 for reproducibility
valid_set25 = df_input25.sample(frac=0.05, random_state=0)

# create a copy of the datasets
training_features25 = training_set25.copy()
valid_features25 = valid_set25.copy()
test_features25 = test_set25.copy()

#use pop to remove the final column (NUM_COLLISIONS), set it to X_labels for the test and training sets
training_labels25 = training_features25.pop('NUM_COLLISIONS')
test_labels25 = test_features25.pop('NUM_COLLISIONS')

#use pop to remove the final column (NUM_COLLISIONS) from valid_features to create validation set
valid_features25.pop('NUM_COLLISIONS')

# divide by the scale factor (1200)
training_labels25 = training_labels25/SCALE_NUM_COLLISIONS
test_labels25 = test_labels25/SCALE_NUM_COLLISIONS

normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features25))

# Normalisation layer (7 inputs), 3 layers of 48, with 1 output.
dnn_model_17 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_17.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

      day  temp  dewp  visib  mxpsd   max  fog  NUM_COLLISIONS
1785    3  62.9  51.9   10.0   15.0  73.9    0             686
2807    7  62.2  56.2    8.8   18.1  66.0    1             206
2476    5  66.9  63.6    7.8   17.1  77.0    1             752
2288    3  59.8  55.0    9.6   15.9  66.0    0             655
1697    2  55.3  48.6   10.0   15.0  64.0    0             720
640     1  32.7  23.9    9.7   11.1  43.0    0             706


In [91]:
%%time
history = dnn_model_17.fit(
    training_features25,
    training_labels25,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 24.3 s, sys: 1.16 s, total: 25.5 s
Wall time: 29.4 s


In [92]:
# evaluate the mean absolute error for the model
dnn_model_17_results = dnn_model_17.evaluate(test_features25, test_labels25, verbose=0)
print(dnn_model_17_results)

# test the model on the validation set and compare predicted with actual
dnn_day_predictions_17 = dnn_model_17.predict(valid_features25)*1200 #scale factor

#create a copy of the validation set
valid_set25a = valid_set25.copy()

#add a column for the predicted number of collisions using our model
valid_set25a["NUM_COLLISIONS_predicted"] = dnn_day_predictions_17.tolist()

#print the dataframe to compare actual and predicted data
print(valid_set25a)


0.1356942057609558
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
      day  temp  dewp  visib  mxpsd   max  fog  NUM_COLLISIONS  \
3727    4  24.9   4.8   10.0   22.9  44.1    0             412   
3834    7  51.9  51.2    2.1   22.9  55.9    1             505   
2859    6  44.9  36.6    9.4   18.1  55.0    0             549   
3822    2  32.6  25.6    8.4   22.9  52.0    0             613   
3578    2  60.5  55.1    8.7   21.0  64.9    0             715   
...   ...   ...   ...    ...    ...   ...  ...             ...   
3537    4  46.6  41.2   10.0   21.0  55.9    0             197   
2914    4  29.9  11.8   10.0   19.0  36.0    0             625   
2765    5  61.7  58.6    4.0   18.1  66.0    1             708   
2248    7  71.5  70.6    7.0   15.9  77.0    1             284   
2010    3  41.1  38.4    6.5   15.9  48.9    0             583   

     NUM_COLLISIONS_predicted  
3727         [313.4541015625]  
3834      [401.6940002441406]  
2859      [526.8675537109375]  
3822      [480.99905395

###Compare Mean Absolute Error

In [93]:
print("Linear Modelling Mean absolute error (MAE) results")
print("All variables: " + str(mean_absolute_error_model_0))
print("Significant variables: " + str(mean_absolute_error_model_1))
print("Significant variables except mean temperature: " + str(mean_absolute_error_model_2))
print("Significant variables except mean dewpoint: " + str(mean_absolute_error_model_3))
print("Significant variables except mean visibility: " + str(mean_absolute_error_model_4))
print("Significant variables except mean wind speed: " + str(mean_absolute_error_model_5))
print("Significant variables except maximum sustained wind speed: " + str(mean_absolute_error_model_6))
print("Significant variables except maximum wind gust: " + str(mean_absolute_error_model_7))
print("Significant variables except day of week: " + str(mean_absolute_error_model_8))
print("")
print("DNN Modelling Mean absolute error (MAE) results")
print("All variables: " + str(dnn_model_1_results))
print("Remove mean temperature: " + str(dnn_model_2_results))
print("Remove mean dewpoint: " + str(dnn_model_3_results))
print("Remove mean sea level pressure: " + str(dnn_model_4_results))
print("Remove maximum temperature: " + str(dnn_model_5_results))
print("Remove minimum temperature: " + str(dnn_model_6_results))
print("Remove reported fog: " + str(dnn_model_7_results))
print("Remove mean wind speed: " + str(dnn_model_8_results))
print("Remove maximum sustained wind speed: " + str(dnn_model_9_results))
print("Remove maximum wind gust: " + str(dnn_model_10_results))
print("Remove total precipitation: " + str(dnn_model_11_results))
print("Remove mean visibility: " + str(dnn_model_12_results))
print("Remove day of week: " + str(dnn_model_13_results))
print("All significant variables from linear model: " + str(dnn_model_14_results))
print("Remove all variables whose removal decreased MAE: " + str(dnn_model_15_results))
print("Remove all variables whose removal decreased MAE AND increase neurons in dense layer: " + str(dnn_model_16_results))
print("Remove all variables whose removal decreased MAE AND increase number of layers: " + str(dnn_model_17_results))

Linear Modelling Mean absolute error (MAE) results
All variables: 0.1782083511352539
Significant variables: 0.1279795616865158
Significant variables except mean temperature: 0.143507719039917
Significant variables except mean dewpoint: 0.13664111495018005
Significant variables except mean visibility: 0.13224492967128754
Significant variables except mean wind speed: 0.13423854112625122
Significant variables except maximum sustained wind speed: 0.1345731019973755
Significant variables except maximum wind gust: 0.1355268508195877
Significant variables except day of week: 0.14146864414215088

DNN Modelling Mean absolute error (MAE) results
All variables: 0.14019443094730377
Remove mean temperature: 0.1428210735321045
Remove mean dewpoint: 0.1623493731021881
Remove mean sea level pressure: 0.1399620622396469
Remove maximum temperature: 0.14132946729660034
Remove minimum temperature: 0.13199076056480408
Remove reported fog: 0.14263370633125305
Remove mean wind speed: 0.13620688021183014
Remo

# Conclusions

It depends on the run, but in this case the lowest mean absolute error was achieved by using the linear model with only the significant variables:
*   mean temperature
*   mean dewpoint
*   mean visibility
*   mean wind speed
*   maximum sustained wind speed
*   maximum wind gust
*   total precipitation
*   day of week

Note that the seed has been set so that the results match the write-up and so removing or changing the seed might produce slightly different results. However, every time I ran these models the lowest mean absolute error was obtained from one of the linear models with either all or some of the significant variables included. Removing any of the significant variables did not greatly reduce the mean absolute error. Prior analysis showed the residuals were not perfectly normally distributed and the mean was not near zero. Therefore, it was thought that a linear model may not be the best predictor and other models, such as DNN should be considered. However, the DNN models failed to produce a lower mean absolute error than the linear models, indicating they are not a better for the data. Increasing either the number of layers or the number of neurons on the best DNN model did not further reduce the mean absolute error. Therefore, the best fitting model remains the linear regression model containing the significant variables listed above.

One possible reason that the DNN model did not perform better than the linear model could be that the amount of data was not large enough for the deep neural network to learn from. It was expected that 12 years worth of data would have been sufficient; however, some data was excluded due to missing values. As a result, the DNN may have overfit the data resulting in a lower performing model.

It is also possible that even though prior analysis showed that the residuals were not perfectly normally distributed and the mean was not that close to zero, these values were approximate enough that a linear regression model best fits the data. Trying to apply a deep neural network to the data may have introduced unnecessary complexity, when in fact the relationship between variables was close enough to linear all along.

Another possibility would be to try looking at other metrics, such as root mean squared error, to compare the models. Mean absolute error was chosen to compare the models due to the number of outliers, but it is possible that other metrics would offer further insight.

In conclusion, the results show that the best model to predict the relationship between weather conditions, day of the week, and number of collisions is the linear model using the following variables:
*   mean temperature
*   mean dewpoint
*   mean visibility
*   mean wind speed
*   maximum sustained wind speed
*   maximum wind gust
*   total precipitation
*   day of week

This model produced the lowest mean absolute error of the models tested and was able to predict the number of collisions based on input variables with reasonable accuracy. It is hoped that by inputting this information, the model can be used by the New York City Emergency Services Unit to predict how many collisions are likely to occur on a particular day. In this way, the model can lead to informed decision-making and facilitate planning and optimization of staff resources by the New York City Emergency Services Unit. It is recommended that this model is used in conjunction with the findings from the prior analysis of collision map data so that resources can be directed to where they are most needed.



# References

Abadi, M. et al. (2015) 'TensorFlow: Large-scale machine learning on heterogeneous systems'. Available at: http://tensorflow.org/

Harris, C.R. et al. (2020) 'Array programming with NumPy'. *Nature*, 585, pp.357-362. Available at: https://doi.org/10.1038/s41586-020-2649-2

McKinney, W. *et al.* (2010) 'Data structures for statistical computing in python'. *Proceedings of the 9th Python in Science Conference*, pp. 51-56. Available at: https://doi.org/10.25080/Majora-92bf1922-00a

New York Police Department (2017). 'NYPD Motor Vehicle Collisions' Available at: https://console.cloud.google.com/marketplace/product/city-of-new-york/nypd-mv-collisions (Accessed: 10 September 2024).

NOAA National Centers of Environmental Information (1999) 'Global Surface Summary of the Day - GSOD'. 1.0. (2012-2024). NOAA National Centers for Environmental Information. Available at: https://console.cloud.google.com/marketplace/product/noaa-public/gsod (Accessed: 10 September 2024).